In [1]:
!pip install optuna
!pip install pmdarima
!pip install pmdarima scipy==1.2 -Uqq
!pip install yellowbrick==0.9.1 scikit-learn==0.22.2
!pip install sktime --ignore-installed
!pip install delayed
!pip install fbprophet
!pip install tbats
!pip install workalendar

import optuna
import os
import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths
# os.chdir("../../..")
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession

import sys, os, time
import matplotlib.cm as cm

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import pandas as pd
from workalendar.core import Calendar
from workalendar.registry import registry
from pandas.tseries.offsets import BDay
import numpy as np
from pandas.tseries.holiday import (
    AbstractHolidayCalendar, Holiday, DateOffset, \
    SU, MO, TU, WE, TH, FR, SA, \
    next_monday, nearest_workday, sunday_to_monday,
    EasterMonday, GoodFriday, Easter
)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error#, mean_absolute_percentage_error


from datetime import datetime
import statistics

import random



from sklearn.preprocessing import OneHotEncoder
from dateutil.relativedelta import relativedelta

import re
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader

spark = SparkSession \
    .builder \
    .appName("WMX - impr forecast - KT") \
    .enableHiveSupport()\
    .config("hive.exec.dynamic.partition", "true")\
    .config("hive.exec.dynamic.partition.mode", "nonstrict")\
    .config("spark.sql.catalogImplementation","hive")\
    .config("spark.sql.hive.convertMetastoreOrc", "false")\
    .config("spark.executor.memory", "3G")\
    .config("spark.driver.memory", "3G")\
    .config("spark.network.timeout", "43200")\
    .config("spark.rdd.compress", "true")\
    .config("spark.cores.max", "512")\
    .config("spark.default.parallelism", "256")\
    .config("spark.dynamicAllocation.minExecutors", "64")\
    .config("spark.dynamicAllocation.maxExecutors", "2048")\
    .config("spark.executor.memoryOverhead", "1G")\
    .config("spark.sql.hive.convertMetastoreOrc", "false")\
    .config("spark.sql.execution.arrow.enabled","true")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.driver.maxResultSize","0")\
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","1g")\
    .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.11.jar')\
    .getOrCreate()

Looking in indexes: https://repository.walmart.com/repository/pypi-proxy/simple/
Looking in indexes: https://repository.walmart.com/repository/pypi-proxy/simple/
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.9.0 requires numpy<=1.19.3, but you have numpy 1.19.5 which is incompatible.
sktime 0.9.0 requires statsmodels<=0.12.1, but you have statsmodels 0.12.2 which is incompatible.
Looking in indexes: https://repository.walmart.com/repository/pypi-proxy/simple/
  Using cached https://repository.walmart.com/repository/pypi-proxy/packages/scikit-learn/scikit_learn-0.22.2-cp36-cp36m-manylinux1_x86_64.whl?originalHref=aHR0cHM6Ly9maWxlcy5weXRob25ob3N0ZWQub3JnL3BhY2thZ2VzL2UxLzdmLzM2NmRjYmExYmEwNzZhODhhNTBiZWE3MzJkYmMwMzNjMGM1YmJmNzg3NjAxMGU2ZWRjNjc5NDg1NzlkNS9zY2lraXRfbGVhcm4tMC4yMi4yLWNwMzYtY3AzNm0tbWFueWxpbnV4MV94ODZfNjQud2hsI3NoYTI1Nj1iYzhiMzdjYTBkZTc5ND

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.9.0 requires numpy<=1.19.3, but you have numpy 1.19.5 which is incompatible.
sktime 0.9.0 requires scikit-learn>=0.24.0, but you have scikit-learn 0.22.2 which is incompatible.
sktime 0.9.0 requires statsmodels<=0.12.1, but you have statsmodels 0.12.2 which is incompatible.
Looking in indexes: https://repository.walmart.com/repository/pypi-proxy/simple/
  Using cached https://repository.walmart.com/repository/pypi-proxy/packages/sktime/sktime-0.9.0-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl?originalHref=aHR0cHM6Ly9maWxlcy5weXRob25ob3N0ZWQub3JnL3BhY2thZ2VzLzczLzcyLzg0MGMzMmI0ZGJhMjM2Y2ZkYmQzZDA0NmJjNTUzZDNlY2ZlNDQwN

  Using cached https://repository.walmart.com/repository/pypi-proxy/packages/threadpoolctl/threadpoolctl-3.1.0-py3-none-any.whl?originalHref=aHR0cHM6Ly9maWxlcy5weXRob25ob3N0ZWQub3JnL3BhY2thZ2VzLzYxL2NmLzZlMzU0MzA0YmNiOWM2NDEzYzRlMDJhNzQ3YjYwMDA2MWMyMWQzOGJhNTFlN2U1NDRhYzdiYzY2YWVjYy90aHJlYWRwb29sY3RsLTMuMS4wLXB5My1ub25lLWFueS53aGwjc2hhMjU2PThiOTlhZGRhMjY1ZmViNjc3MzI4MGRmNDFlZWNlN2IyZTY1NjFiNzcyZDIxZmZkNTJlMzcyZjk5OTAyNDkwN2I= (14 kB)
  Using cached https://repository.walmart.com/repository/pypi-proxy/packages/scipy/scipy-1.5.4-cp36-cp36m-manylinux1_x86_64.whl?originalHref=aHR0cHM6Ly9maWxlcy5weXRob25ob3N0ZWQub3JnL3BhY2thZ2VzL2M4Lzg5LzYzMTcxMjI4ZDVjZWQxNDhmNWNlZDUwMzA1Yzg5ZTg1NzZmZmM2OTVhOTBiNThmZTViYjYwMmI5MTBjMi9zY2lweS0xLjUuNC1jcDM2LWNwMzZtLW1hbnlsaW51eDFfeDg2XzY0LndobCNzaGEyNTY9MzY4YzBmNjlmOTMxODYzMDllMWI0YmViOGUyNmQ1MWRkNmY1MDEwYjc5MjY0YzBmMWU5Y2EwMGNkOTJlYThjOQ== (25.9 MB)
  Using cached https://repository.walmart.com/repository/pypi-proxy/packages/patsy/patsy-0.5.3-py2.py3-none-an

  Using cached https://repository.walmart.com/repository/pypi-proxy/packages/scipy/scipy-1.2.3-cp36-cp36m-manylinux1_x86_64.whl?originalHref=aHR0cHM6Ly9maWxlcy5weXRob25ob3N0ZWQub3JnL3BhY2thZ2VzL2M3L2Q2LzY2MWNmNGZiMzJlY2RhNDBmYzAzNjg1ZWYwOTk1ZGIzZDMzMDBjNjE5ZjQ2MTc1MTYyMWY2NDZhYjNmNi9zY2lweS0xLjIuMy1jcDM2LWNwMzZtLW1hbnlsaW51eDFfeDg2XzY0LndobCNzaGEyNTY9YzA4MDEzZTBmZTE1NTQzNzJkYTkzMTJkNWJhZDU4ODQwMmY3MWMwNjM2ZjBmODZhOWI5YjYxZDUwN2M1OWJhYw== (24.8 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.4
    Uninstalling scipy-1.5.4:
      Successfully uninstalled scipy-1.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.9.0 requires numpy<=1.19.3, but you have numpy 1.19.5 which is incompatible.
sktime 0.9.0 requires statsmodels<=0.12.1, but you have statsmodels 0.12.2 which is incompatible.
Looking in indexes: https://repository.wa

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))
import os
import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths
# os.chdir("../../..")
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession

import sys, os, time
import matplotlib.cm as cm

import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error#, mean_absolute_percentage_error


from datetime import datetime
import statistics

import random



from sklearn.preprocessing import OneHotEncoder
from dateutil.relativedelta import relativedelta

import re
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader

import os
import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths
# os.chdir("../../..")
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os,sys
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
import re
import os
import sys, os, time
import matplotlib.cm as cm

import matplotlib.pyplot as plt

import matplotlib.ticker as mticker

import torch
import torch.nn as nn

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error#, mean_absolute_percentage_error


from datetime import datetime
import statistics

import random
import time
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from dateutil.relativedelta import relativedelta


import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np


import time
from torch.optim.lr_scheduler import ExponentialLR

import sys, os, time
import matplotlib.cm as cm

import torch
import torch.nn as nn

from datetime import datetime
import statistics

import random
import time
import optuna
from optuna.trial import TrialState
import torch.nn.functional as F

import os,sys
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from dateutil.relativedelta import relativedelta
import re
import os
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from dateutil.relativedelta import relativedelta
import torch
from sklearn import preprocessing
from dateutil.relativedelta import relativedelta
# from ray import tune
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
#le = preprocessing.LabelEncoder()
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from dateutil.relativedelta import relativedelta
# from ray import tune
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

cuda:0
1
Tesla V100-SXM2-16GB


In [3]:
import pmdarima as pmd
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
#import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
import sktime
import delayed
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.ets import AutoETS
from warnings import simplefilter

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    MultiplexForecaster,
    TransformedTargetForecaster,
    make_reduction,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.forecasting.theta import ThetaForecaster
from statsmodels.tsa.api import STLForecast
from sktime.forecasting.bats import BATS
from sktime.forecasting.tbats import TBATS
from sktime.forecasting.croston import Croston
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt



In [4]:

from __future__ import print_function
import sys, os, time
import matplotlib.cm as cm

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd

import torch
import torch.nn as nn

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error#, mean_absolute_percentage_error


from datetime import datetime
import statistics

import random
import time

from tqdm.auto import tqdm

# import darts
# from darts import TimeSeries
# from darts.metrics import *
# from darts.utils.missing_values import fill_missing_values

import warnings
from pyspark.sql import Window
import pyspark.sql.functions as f

import os,sys
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
import re
import os
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from dateutil.relativedelta import relativedelta
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import torch
from sktime.forecasting.fbprophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [5]:

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np




In [6]:

from __future__ import print_function
import sys, os, time
import matplotlib.cm as cm

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd

import torch
import torch.nn as nn

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error#, mean_absolute_percentage_error


from datetime import datetime
import statistics

import random
import time

from tqdm.auto import tqdm

# import darts
# from darts import TimeSeries
# from darts.metrics import *
# from darts.utils.missing_values import fill_missing_values

import warnings
from pyspark.sql import Window
import pyspark.sql.functions as f

import os,sys
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
import re
import os
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from dateutil.relativedelta import relativedelta
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import torch


In [7]:
class dataprep:
    def __init__(self, df, input_window=90, output_window=30, stride=1, start='2020-11-01'):
        # le = preprocessing.LabelEncoder()
        df['ds'] = pd.to_datetime(df['ds'], format='%Y-%m-%d')
        start = pd.to_datetime(start, format='%Y-%m-%d') #+ relativedelta(months=months)
        end = start + relativedelta(days=365 + 30) #+ relativedelta(months=months)
        # start1=df.ds.sort_values()[0]
        # end1=df.ds.sort_values()[len(df_panda)-1]
        # if end1<end:end=end1
        # if start1>start:start=start1
        # print(start,end)
        df = df[(df.ds >= start) & (df.ds <= end)]


        df['query_idx'] = df['query_idx'].map(str)
        df.sort_values('ds', inplace=True)
        if df['ds'].iloc[-1]!=end:
            last_date=df.iloc[-1,:]
            last_date['ds']=end
            last_date['daily_supply'] = 0
            df = df.append(last_date)
        if df['ds'].iloc[0]!=start:
            first_date=df.iloc[0,:]
            first_date['ds']=end
            first_date['daily_supply'] = 0
            df = df.append(first_date)
        df.sort_values('ds', inplace=True)
        df['grp'] = df.apply(
            lambda x: '%s_%s_%s_%s' % (x['group_id'], x['query_idx'], x['SearchKeyValue'], x['LocationKeyValue']),
            axis=1)
        dts = (end - start).days + 1

        #qr = df.groupby(['grp'])['daily_supply'].count()
        # self.df_test=df[['ds','daily_supply','grp']]
###############################
        #qrs = list(qr[(dts - qr) <= 50].index)

        #df = df.loc[df['grp'].isin(qrs)]
###############################

        df.sort_values('ds', inplace=True)
        # df.set_index(['ds', 'grp'], append=True,inplace=True)
        # df= df.unstack(fill_value=0)\
        # df=df.asfreq('D', fill_value=0).stack(). \
        #     sort_index(level=1).reset_index()


        try:
            df = df.set_index(['ds', 'grp']).unstack(fill_value=0). \
            asfreq('D', fill_value=0).stack(). \
            sort_index(level=1).reset_index()
        except:
            df = df.groupby(['ds', 'grp']).agg({'daily_supply':['sum'],'group_id':['first'],'query_idx':['first'], 'SearchKeyValue':['first'], 'LocationKeyValue':['first']})
            df = df.unstack(fill_value=0). \
                asfreq('D', fill_value=0).stack(). \
                sort_index(level=1).reset_index()
            df.columns = df.columns.get_level_values(0)
        #cls = ['year', 'month', 'week', 'quarter', 'dayofweek', 'isWeekend', 'Trend', 'weeklyChange', 'priceMov',
        #       'Season', 'isHoliday', 'isPromotion', 'dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
        df.sort_values('ds', inplace=True)
        #df[cls] = df[cls].replace(to_replace=0, method='ffill')  # **********
        #df[cls] = df[cls].replace(to_replace=0, method='bfill')
        self.df = df
        self.test_split_date = pd.to_datetime((end + relativedelta(days=-30)).strftime('%Y-%m-%d'))
        ####################################################################################
        self.val_split_date = pd.to_datetime((self.test_split_date + relativedelta(days=-30)).strftime('%Y-%m-%d'))

        self.df = self.df.drop(['group_id', 'query_idx', 'SearchKeyValue', 'LocationKeyValue'], axis=1)

        self.df_trains = self.df[self.df.ds <= self.val_split_date]
        self.df_tests = self.df[self.df.ds > self.test_split_date]
        df_train_temp = self.df[self.df.ds <= self.test_split_date]

        self.df_test_org = pd.pivot_table(self.df_tests, values='daily_supply', index=['ds'], columns=['grp'])
        self.df_train_org = pd.pivot_table(df_train_temp, values='daily_supply', index=['ds'],columns=['grp'])  # ,aggfunc=np.sum

        self.df_train_org.sort_index(inplace=True)
        # print(self.df_train_org.iloc[:,0])
        # print(self.df_train_org.iloc[-14:,0])

        # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        self.q_train = self.df_train_org.iloc[-14:, :].sum(axis=0) / np.sum(np.sum(self.df_train_org.iloc[-14:, :]))
        self.q_test = self.df_test_org.sum(axis=0) / np.sum(np.sum(self.df_test_org))
        del self.df_train_org
        del self.df_trains
        del self.df_tests
        del df_train_temp
        self.df = self.df.groupby('ds').agg(
            {'daily_supply': 'sum'
             #,'shifted_annual_daily_supply': 'sum', 'year': lambda x: x.iloc[0],
             #'month': lambda x: x.iloc[0],
             #'week': lambda x: x.iloc[0], 'quarter': lambda x: x.iloc[0], 'dayofweek': lambda x: x.iloc[0],
             #'isWeekend': lambda x: x.iloc[0], 'Trend': lambda x: x.value_counts().index[0], 'Trend_Var': 'sum',
             #'weeklyChange': lambda x: x.value_counts().index[0], 'priceMov': lambda x: x.value_counts().index[0],
             #'Season': lambda x: x.iloc[0],
             #'isHoliday': lambda x: x.iloc[0], 'isPromotion': lambda x: x.iloc[0], 'dayofyear_sin': lambda x: x.iloc[0],
             #'dayofyear_cos': lambda x: x.iloc[0], 'month_sin': lambda x: x.iloc[0], 'month_cos': lambda x: x.iloc[0]
             })

        self.inp = input_window
        self.out = output_window
        self.stride = stride

    def _get_data(self):
        # strCol = ['year', 'month', 'week', 'quarter', 'dayofweek', 'isWeekend', 'Trend', 'weeklyChange', 'priceMov',
        #           'Season', 'isHoliday',
        #           'isPromotion']  # *******************************************************************
        # for cols in strCol:
        #     encoder = OneHotEncoder()
        #     encoder_df = pd.DataFrame(encoder.fit_transform(self.df[[cols]]).toarray())
        #     column_name = encoder.get_feature_names([cols])
        #     encoder_df.columns = column_name
        #     encoder_df.index = self.df.index
        #     self.df = self.df.join(encoder_df)
        #     self.df = self.df.drop(cols, axis=1)

        Cols_minmax = ['daily_supply',
                       # 'shifted_annual_daily_supply', 'Trend_Var', 'dayofyear_sin', 'dayofyear_cos',
                       # 'month_sin', 'month_cos'
                       ]
        for cols in Cols_minmax:
            if cols != 'shifted_annual_daily_supply':
                self.scaler_fz = MinMaxScaler(feature_range=(-1, 1))
                self.scaler_fz = self.scaler_fz.fit(self.df[cols].values.reshape(-1, 1))
            if cols == 'daily_supply':
                self.scaler_f = self.scaler_fz
            self.df[cols] = self.scaler_fz.transform(self.df[cols].values.reshape(-1, 1))

        self.df_train = self.df[self.df.index <= self.val_split_date]
        self.df_val = self.df[(self.df.index > self.val_split_date) & (self.df.index <= self.test_split_date)]
        self.df_test = self.df[self.df.index > self.test_split_date]

        train_date, test_date = list(self.df_train.index), list(self.df_test.index)

        average = 0

        try: val_shape = self.df_val['daily_supply'].shape[0]
        except:val_shape = self.df_val.shape[0]
        self.df_train.reset_index(inplace=True, drop=True)
        self.df_val.reset_index(inplace=True, drop=True)
        self.df_test.reset_index(inplace=True, drop=True)


        if self.inp > val_shape:
            df_test = np.concatenate([self.df_train.iloc[-(self.inp - val_shape):], self.df_val, self.df_test], axis=0)
            df_val = np.concatenate([self.df_train.iloc[-self.inp:], self.df_val], axis=0)
        else:
            df_test = np.concatenate([self.df_val.iloc[:, :][-self.inp:], self.df_test], axis=0)#$$$$$$need to change for more features
            df_val = np.concatenate([self.df_train.iloc[-self.inp:], self.df_val], axis=0)#$$$$$$need to change for more features
        self.df_test, self.df_val = df_test, df_val
        df_test = pd.DataFrame(df_test)
        try:df_test.columns = self.df_train.columns
        except:df_test.name = self.df_train.name

        df_val = pd.DataFrame(df_val)
        try:df_val.columns = self.df_train.columns
        except:df_val.name = self.df_train.name

        self.df_val, self.df_test = df_val, df_test

        X_train, Y_train = self.sliding_windows(self.df_train, seq_length_x=self.inp, seq_length_y=self.out,
                                                time_step=self.stride)
        X_val, Y_val = self.sliding_windows(self.df_val, seq_length_x=self.inp, seq_length_y=self.out,
                                            time_step=self.stride)
        X_test, Y_test = self.sliding_windows(self.df_test, seq_length_x=self.inp, seq_length_y=self.out,
                                              time_step=self.stride)
        return X_train, Y_train, X_val, Y_val, X_test, Y_test, self.df_test_org.values, self.q_train, self.q_test

    def sliding_windows(self, data, seq_length_x, seq_length_y, time_step):
        x = []
        y = []
        for i in range(0, len(data) - seq_length_x - seq_length_y + 1, time_step):
            try:
                _x = data.iloc[i:(i + seq_length_x), :].values
                _y = data.loc[i + seq_length_x:i + seq_length_x + seq_length_y - 1,'daily_supply'].values
            except:
                _x = data.iloc[i:(i + seq_length_x)].values
                _y = data.loc[i + seq_length_x:i + seq_length_x + seq_length_y - 1].values
              # data.loc[i + seq_length_x:i + seq_length_x + seq_length_y,'daily_supply'].values
            x.append(_x)
            y.append(_y)
        return np.array(x), np.array(y)


In [8]:
class Data(Dataset):
    def __init__(self, x, y):
        np.random.seed(int(43))
        super(Data, self).__init__()
        # store the raw tensors
        self._x = x
        self._y = y

    def __len__(self):
    # a DataSet must know it size
        return self._x.shape[0]

    def __getitem__(self, index):
        x = self._x[index, :]
        y = self._y[index, :]
        return x, y
    
class LSTM(nn.Module):

    def __init__(self, output_size, input_size=1, hidden_size=4, num_layers=1, dropout=.3,relu='False'):
        super(LSTM, self).__init__()

        self.output_size = output_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.relu=relu
        # self.dp = nn.Dropout(dropout)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=dropout)  # .25
        # self.batchnorm=nn.BatchNorm1d(hidden_size)
        self.rel=nn.LeakyReLU(hidden_size)
        # self.fc = nn.Linear(hidden_size, hidden_size)
        # self.dp = nn.Dropout(.5)
        self.fc=nn.Linear(hidden_size, output_size)
    def forward(self, x):
        for it in self.lstm.parameters():
            w = it
            break

        h_0 = w.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w.requires_grad).to(self.device)

        c_0 = w.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w.requires_grad).to(self.device)

        # Propagate input through LSTM

        h_out, _ = self.lstm(x, (h_0, c_0))
        #h_out.requires_grad_()
        h_out = h_out[:, -1, :]
        h_out = h_out.reshape(-1, self.hidden_size)
        #out = self.dp(h_out)

        #out = self.batchnorm(h_out)
        if self.relu=='True':
            h_out=  self.rel(h_out)
        # out=  self.fc(h_out)
        # out = self.dp(out)
        out = self.fc(h_out)
        #out.requires_grad_()
        return out

class LSTM1(nn.Module):

    def __init__(self, output_size, input_size=1, hidden_size=4, num_layers=1,dropout=.1):
        super(LSTM, self).__init__()

        self.output_size = output_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,dropout = .2)
        self.batchnorm=nn.BatchNorm1d(hidden_size)
        self.dp=nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x):
        for it in self.lstm.parameters():
            w = it
            break

        h_0 = w.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w.requires_grad).to(self.device)

        c_0 = w.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w.requires_grad).to(self.device)

        # Propagate input through LSTM

        h_out, _ = self.lstm(x, (h_0, c_0))
        h_out.requires_grad_()
        h_out = h_out[:, -1, :]
        h_out = h_out.reshape(-1, self.hidden_size)
        out=self.batchnorm(h_out)
        out=self.dp(out)
        out = self.fc(out)

        out.requires_grad_()
        return out
    
    

class LSTM3(nn.Module):

    def __init__(self, output_size, input_size=1, hidden_size=4, num_layers=1,dropout=.1):
        super(LSTM, self).__init__()

        self.output_size = output_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,dropout = dropout)
        self.lstm2 = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,dropout = dropout)
        self.batchnorm=nn.BatchNorm1d(hidden_size)
        self.dp=nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x):
        for it in self.lstm1.parameters():
            w = it
            break

        h_0 = w.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w.requires_grad).to(self.device)

        c_0 = w.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w.requires_grad).to(self.device)
        for it in self.lstm2.parameters():
            w1 = it
            break
        h_1 = w1.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w1.requires_grad).to(self.device)

        c_1 = w1.new_zeros(
            self.num_layers, x.size(0), self.hidden_size, requires_grad=w1.requires_grad).to(self.device)

        torch.nn.init.xavier_normal_(h_0)
        torch.nn.init.xavier_normal_(c_0)
        torch.nn.init.xavier_normal_(h_1)
        torch.nn.init.xavier_normal_(c_1)
        # Propagate input through LSTM

        h_out, _ = self.lstm1(x, (h_0, c_0))
        h_out.requires_grad_()
        #h_out = h_out[:, -1, :]
        #h_out = h_out.reshape(-1, self.hidden_size)
        
        h_out1, _ = self.lstm2(h_out, (h_0, c_0))
        h_out1.requires_grad_()
        h_out1 = h_out1[:, -1, :]
        h_out1 = h_out1.reshape(-1, self.hidden_size)
        
        out=self.batchnorm(h_out1)
        out=self.dp(out)
        out = self.fc(out)

        out.requires_grad_()
        return out


In [9]:
def SMAPE(A, F):
    smape_mean = 0
    for i in range(A.shape[0]):
        if A[i] == F[i]:
            continue
        smape_mean += 2 * np.abs(F[i] - A[i]) / (np.abs(A[i]) + np.abs(F[i]))
    smape_mean = 1 / A.shape[0] * smape_mean
    return smape_mean
def SMAPE2(A, F):
    smape_mean = 0
    for i in range(A.shape[0]):
        if A[i] == F[i]:
            continue
        smape_mean += 2 * np.abs(F[i] - A[i]) / (np.abs(A[i]) + np.abs(F[i]))
    smape_mean = 1 / A.shape[0] * smape_mean
    return smape_mean
def WAPE(A, F):
    return np.sum((np.abs(F - A))/np.sum(A))
def weightedSMAPE(A,F,wq):
    res=0
    print('number of smape: ',len(wq))
    for i in range(len(wq)):
        res+=wq[i]*SMAPE(A[:,i],F[:,i])
    return res
def WAPE_fun(actual,predicted,w):
    #w=query percentage in the cluster
    wape_tot=0
    #print('number of smape: ', len(w))
    for i in range(len(w)):
        if np.sum(actual[:,i].reshape(-1,1))<=0:
            continue
        wape_tot+=w[i]*WAPE(actual[:,i].reshape(-1,1),predicted[:,i].reshape(-1,1))
    return wape_tot
def PE(A, F):
    return np.abs(np.sum(F)-np.sum(A))/np.sum(A)
def PE_fun(actual,predicted,w):
    wape_tot = 0
    for i in range(len(w)):
        if np.sum(actual[:, i].reshape(-1, 1)) <= 0:
            continue
        wape_tot += w[i] * PE(actual[:, i].reshape(-1, 1), predicted[:, i].reshape(-1, 1))
    return wape_tot
def estimate(actual,pred,q):
    if type(q)==pd.core.series.Series:q=q.values
    #pred=pred * q.reshape(-1, )
    smape_error = weightedSMAPE(actual, pred, q)
    wape=WAPE_fun(actual,pred,q)
    pe=PE_fun(actual,pred,q)
    return smape_error,wape,pe
def metric_dist(A,F,q,metric='wape'):
    res=[]
    if metric=='wape':
        for i in range(len(q)):
            if np.sum(A[:, i].reshape(-1, 1)) <= 0:
                continue
            res.append(WAPE(A[:, i].reshape(-1, 1), F[:, i].reshape(-1, 1)))
    elif metric=='smape':
        for i in range(len(q)):
            if np.sum(A[:, i].reshape(-1, 1)) <= 0:
                continue
            res.append(SMAPE(A[:, i].reshape(-1, 1), F[:, i].reshape(-1, 1)))
    else:
        for i in range(len(q)):
            if np.sum(A[:, i].reshape(-1, 1)) <= 0:
                continue
            res.append(PE(A[:, i].reshape(-1, 1), F[:, i].reshape(-1, 1)))
    return [np.percentile(res,50),np.percentile(res,80),np.percentile(res,90)]
def Estimate(actual,pred,metric='wape'):
    if metric=='pe':return PE(actual,pred)
    elif metric=='wape':return WAPE(actual.reshape(-1,1),pred.reshape(-1,1))
    elif metric=='smape':return SMAPE(actual.reshape(-1,1),pred.reshape(-1,1))
    else:
        raise ValueError('incorrect metric is defined')
def myloss(A,F):
    #loss=torch.mean(np.abs((actual.detach().numpy() - predict.detach().numpy()) / actual))
    loss=torch.mean(1 / A.size(dim=0)* torch.sum(2 * torch.abs(F - A) / (torch.abs(A) + torch.abs(F))))
    return loss#torch.from_numpy(np.asarray(loss))

In [10]:
def train_lstm(config,save_model=False):
    num_epochs = config['num_epochs']
    learning_rate = config['learning_rate']
    window_size = 30
    output_size = 30
    relu=config['relu']
    input_size = train_set_in.shape[2]
    hidden_size = config['hidden_size']
    num_layers = 1
    drop_out = config['dropout']
    batch_size=config['batch_size']
    gamma=config['gamma']
    optimizer_name=config['optimizer_name']
    num_layers=config['num_layer']
    #dropout1=config['dropout1']
    lstm = trainLSTM(num_epochs, learning_rate, output_size, input_size, hidden_size, num_layers, drop_out,gamma,optimizer_name,relu)
    lstm.train(train_set_in, train_set_out,val_set_in,val_set_out,test_in_set,test_out_set,batch_size=batch_size)
    val_pred,_, valid_smape, valid_wape,valid_pe= lstm.valid(val_set_in, val_set_out)
    testing_predict, testing_truth, Wsmape, Wape, Wpe, smape_dis, wape_dis, pe_dis, smape_agg, wape_agg,pe_agg = lstm.valid(test_in_set, test_out_set,not_test=False,pred_is_req=True,df_test_org=df_test_org)
    print('for sanity check', valid_smape[0], valid_wape,valid_pe, 'the mean is: ',(valid_smape[0]+valid_wape+valid_pe)/3)
    return [Wsmape, Wape, Wpe, smape_dis, wape_dis, pe_dis, smape_agg, wape_agg,pe_agg,lstm,val_pred.reshape(-1,),valid_wape]#[training_smape*100,valid_smape*100, testing_smape*100,training_wape*100,valid_wape*100, testing_wape*100,lstm]
def running_lstm(config,save_model=False,Device='cpu'):
    num_epochs = config['num_epochs']
    learning_rate = config['learning_rate']
    window_size = 30
    output_size = 30
    batch_size=config['batch_size']
    input_size = train_set_in.shape[2]
    hidden_size = config['hidden_size']
    num_layers = 1
    drop_out=config['dropout']
    gamma=config['gamma']
    optimizer_name=config['optimizer_name']
    num_layers=config['num_layer']
    relu=config['relu']
    #dropout1=config['dropout1']
    lstm = trainLSTM(num_epochs, learning_rate, output_size, input_size, hidden_size, num_layers,drop_out,gamma,optimizer_name,relu)
    #lstm.to(Device)
    loss=lstm.train(train_set_in, train_set_out,val_set_in,val_set_out,test_in_set,test_out_set,batch_size=batch_size)
    _,_, Wsmape, Wwape,Wpe = lstm.valid(val_set_in, val_set_out)
    #testing_predict, testing_truth, testing_smape,testing_wape,smape_agg,wape_agg = lstm.valid(val_set_in, val_set_out)
    return Wsmape[0],Wwape,Wpe

In [11]:
# # # Tail clusters
test_group_id_list = [
 '/1005862/1007220/1044020/3392907',
 '/3944/77622/8375901/1230415/1230417',
 '/4171/645779/2165088/3723286',
 '/1334134/5899871/7445675/5456260',
 '/5428/1102183/8617474/1564368',
 '/2637/5249422/5222147',
 '/1334134/6355365/1285843/3783885',
 '/976759/976787/1001390/2523039/2119183',
 '/91083/2962826/8564995/2827548',
 '/4125/4134/1026285/9642310/1251682',
 '/1005862/1007220/1007399/6461098',
 '/5427/133283/9095197',
 '/1115193/1071967/1149380/5135111',
 '/976759/976794/5403011/9731488',
 '/4125/4134/2350251/1078395',
 '/5428/8018905/5374562/7304919/6954191',
 '/6197502/5702707/9458810/9028847',
 '/91083/1074769/6350088',
 '/5440/1001299/6148116',
 '/1334134/6172404/7659444/3836217',
 '/3920/1987289/582374/8324310/1974555',
 '/4125/4161/4165/1075707',
 '/976760/1414629/6578740/1310637',
 '/91083/1074765/9183778/3753296',
 '/3920/582053/584177/585913',
 '/976760/1005863/8121112',
 '/1105910/133161/1072306/5297453/5115891',
 '/976760/1005860/4157476/8398146/6630246',#
 '/2637/6749799/2053705',
 '/1072864/1067612/6801317/3181011/4259773',
 '/1085666/5859906/3311357',
 '/976760/9575239/2981737',
 '/2637/9489599/2275107',
 '/4125/4134/1078384/1078388',
 '/5440/202073/1749780/3053202/2610736',
 '/1105910/133161/1103334/3123349/9119436',
 '/1334134/6172404/4854691/2291474',
 '/4171/7357538/6711451/4147327',
 '/1334134/8495017/8541719',
 '/4125/546956/4128/4547850/2751930',
 '/1072864/1067618/1230774/8250798/6489704',
 '/1072864/1808984/4037404/9537015',#
 '/1085666/3316357/6852893/3607949',
 '/1229749/6562796/6709176',
 '/91083/1077064/8595272/2238009',#
 '/5440/9672181',
 '/1072864/1067618/1230774/8250798/6489704',
 '/1085666/3147628/8028415/6121950',#
 '/4044/103150/4038/7389568/2455597',#
 '/976760/1876667/7219872']

In [12]:
start_month=['2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01',
            '2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01']
name=['September','October','November','December','January','February','March','April','May','June','July','August']

In [13]:
for i in range(len(name)):
    print(name[i],' : ',start_month[i])

September  :  2020-09-01
October  :  2020-10-01
November  :  2020-11-01
December  :  2020-12-01
January  :  2021-01-01
February  :  2021-02-01
March  :  2021-03-01
April  :  2021-04-01
May  :  2021-05-01
June  :  2021-06-01
July  :  2021-07-01
August  :  2021-08-01


In [14]:
# current_date = '20220831'
# test_group_id = test_group_id_list[0]#'/3944/1060825/447913'
# # Path to the new data
# path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
# # read in data
# tf_data_pd = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous

# # The new train+valid test split logic becomes (if you use the same logic - past year years of data as train and test and predict the next 30 days)
# # Month | train+valid | test
# # Sep | 2020.09 - 2021.08 | 2021.09
# # Start date is 2020-09-01 and next 365 days is the train+valid and the following 30 days is the test. For the next "month"'s eval model, we will move the entire train+valid+test for the next 30 days.

In [15]:
#tf_data_pd.to_csv('df_1_noF.csv', index=False)

In [16]:
def objective(trial):
    params={
        #"num_layers":trial.suggest_int("num_layer",1,2),
        'optimizer_name':trial.suggest_categorical("optimizer_name", ["Adam",'RMSprop','AdamW','SGD']),#"Adam",'RMSprop',
        "hidden_size":trial.suggest_int('hidden_size',8,20),
        'dropout':trial.suggest_uniform("dropout",0,0.10),
        #'dropout1':trial.suggest_uniform("dropout1",0,0.3),
        'learning_rate':trial.suggest_loguniform('learning_rate',1e-4,1e-0),
        'num_epochs':trial.suggest_int('num_epochs',100,100),
        'batch_size':trial.suggest_int('batch_size',32,64),
        'gamma':trial.suggest_uniform("gamma",0.8,1),
        'num_layer':trial.suggest_int('num_layer',1,3),
        'relu':trial.suggest_categorical('relu',['True','False'])}
    temp_loss=running_lstm(params,save_model=False)
    return temp_loss


In [17]:
class trainLSTM:
    def __init__(self, num_epochs, learning_rate, output_size, input_size, hidden_size, num_layers, drop_out, gamma,
                 optimizer_name, relu=False):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.output_size = output_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.drop_out = drop_out
        self.lstm = LSTM(output_size, input_size, hidden_size, num_layers, drop_out, relu)
        self.lstm.to(self.device)
        self.criterion = torch.nn.MSELoss()  # mean-squared error for regression: MSELoss  #L1Loss
        self.gamma = gamma
        self.optimizer_name = optimizer_name
        self.optimizer = getattr(torch.optim, optimizer_name)(self.lstm.parameters(), lr=learning_rate)
        self.val_loss_ep = []
        self.train_loss_ep = []
        self.test_loss_ep = []
        # torch.optim.Adam(self.lstm.parameters(), lr=learning_rate)

    def train(self, trainX, trainY, valX, valY, testX, testY, batch_size=32, n_epochs_stop=10):
        # Train the model
        # self.lstm.train()
        trainX, trainY = trainX.to(self.device), trainY.to(self.device)
        valX, valY = valX.to(self.device), valY.to(self.device)
        DF = Data(trainX, trainY)
        trainloader = DataLoader(dataset=DF, batch_size=batch_size, shuffle=True, num_workers=0)  ####
        # Early stopping
        last_loss = 1e10
        patience = n_epochs_stop
        trigger_times = 0
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min', patience=75, factor=self.gamma)  #ExponentialLR(self.optimizer, gamma=self.gamma)#
        clip = 5
        PATH = "best_mod_agg.pt"
        for epoch in range(self.num_epochs):
            train_losses = 0
            self.lstm.train()
            start1 = time.time()
            for x, y in trainloader:
                # trainX.requires_grad_()
                # print(x[0,:5,:])
                outputs = self.lstm(x)

                # obtain the loss function
                loss = self.criterion(outputs.reshape(-1, 1),
                                      y.reshape(-1, 1))  # myloss(outputs.reshape(-1, 1), y.reshape(-1, 1))#.
                # loss.requires_grad_()
                self.optimizer.zero_grad()
                loss.backward()  # retain_graph=True
                # nn.utils.clip_grad_norm_(self.lstm.parameters(), clip)
                self.optimizer.step()
                train_losses += loss.item()  # *x.size(0)
            train_losses = train_losses / len(DF)
            # if epoch % 5 == 0:
            #    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item() / batch_size))
            self.train_loss_ep.append(train_losses)

            current_loss = self.eval(valX, valY)  # Estimate(valY.data.cpu().numpy(), output.data.cpu().numpy())
            scheduler.step(current_loss)
            self.val_loss_ep.append(current_loss)
            test_loss = self.eval(testX, testY)
            self.test_loss_ep.append(test_loss)
            if current_loss > last_loss and epoch >= 10:
                trigger_times += 1
                # print('Trigger Times:', trigger_times)
                if trigger_times >= patience:
                    print('Early stopping!\nStart to test process.', epoch)
                    break
            else:
                trigger_times = 0
                # torch.save({
                #     'epoch': epoch,
                #     'model_state_dict': self.lstm.state_dict(),
                #     'optimizer_state_dict': self.optimizer.state_dict(),
                #     'loss': train_losses,
                #     'val_loss':current_loss
                # }, PATH)
                # print(self.lstm.state_dict()['lstm.bias_hh_l4'][:5])
                torch.save({'model': self.lstm, 'val_loss': current_loss}, PATH)
                # print(self.lstm.state_dict())
            last_loss = current_loss
            # print('Epoach : ', epoch, ' train_loss: ', train_losses, ' valid_loss: ', current_loss, 'test loss',
            #      test_loss, 'time taken', time.time() - start1)
        checkpoint = torch.load(PATH)
        self.lstm = checkpoint['model']
        # self.lstm.load_state_dict(checkpoint['model_state_dict'])
        # self.lstm=LSTM(self.output_size, self.input_size, self.hidden_size, self.num_layers, self.drop_out)
        # self.lstm.load_state_dict(checkpoint['model_state_dict'])
        # self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        # print('lets check the model poaram')
        # print(self.lstm.state_dict()['lstm.bias_hh_l4'][:5])
        current_loss = checkpoint['val_loss']
        return current_loss

    def eval(self, valX, valY, batch_size=32):
        valX, valY = valX.to(self.device), valY.to(self.device)
        DFv = Data(valX, valY)
        val_loader = torch.utils.data.DataLoader(dataset=DFv, batch_size=batch_size, shuffle=False)
        val_loss = 0
        self.lstm.eval()  # prep model for evaluation
        with torch.no_grad():
            for x, y in val_loader:
                outputs = self.lstm(x)
                loss = self.criterion(y, outputs)  # Estimate(valY.data.cpu().numpy(), outputs.data.cpu().numpy())
                val_loss += loss.item()  # *batch_size#*x.size(0)
            current_loss = val_loss / len(DFv)
        return current_loss

    def valid(self, testX, testY, not_test=True, pred_is_req=False, df_test_org=None):
        # evaluate the model
        self.lstm.eval()
        testX, testY = testX.to(self.device), testY.to(self.device)
        test_predict = self.lstm(testX)
        data_predict = test_predict.data.cpu().numpy()
        dataY_plot = testY.data.cpu().numpy()  # to(self.device)
        data_predict1 = prep.scaler_f.inverse_transform(data_predict.T)
        try:
            dataY_plot = prep.scaler_f.inverse_transform(dataY_plot.T)
        except:
            dataY_plot = dataY_plot.reshape(dataY_plot.shape[0], dataY_plot.shape[1])
            dataY_plot = prep.scaler_f.inverse_transform(dataY_plot.T)
        data_predict1[data_predict1 < 0] = 0
        if not_test:
            smape, wape, pe = Estimate(dataY_plot, data_predict1, metric='smape'), Estimate(dataY_plot, data_predict1, metric='wape'), Estimate(dataY_plot, data_predict1, metric='pe')
            #print('wape is :', smape, wape, pe)
            return data_predict1[:,-1], dataY_plot, smape, wape, pe
        else:
            smape_agg, wape_agg, pe_agg = Estimate(dataY_plot, data_predict1, metric='smape'), Estimate(dataY_plot,data_predict1, metric='wape'), Estimate(
                dataY_plot, data_predict1, metric='pe')
            #print('smape is: ', smape_agg, 'wape is: ', wape_agg, 'pe is:', pe_agg)
            data_predict11 = np.tile(data_predict1.T, (q.shape[0], 1)).T
            data_predict2 = data_predict11 * q.values.reshape(-1, 1).T
            Wsmape, Wwape, Wpe = estimate(df_test_org, data_predict2, wq)
            smape_dis, wape_dis, pe_dis = metric_dist(df_test_org, data_predict2, wq, metric='smape'), metric_dist( df_test_org, data_predict2, wq, metric='wape'), metric_dist(df_test_org, data_predict2, wq, metric='pe')
#         if pred_is_req:
#             data_predict_2 = data_predict.T
#             data_predict_2 = prep.scaler_f.inverse_transform(data_predict_2)
#             data_predict_2[data_predict_2 < 0] = 0
        return data_predict2, dataY_plot, Wsmape, Wwape, Wpe, smape_dis, wape_dis, pe_dis, smape_agg[
            0], wape_agg, pe_agg


In [18]:
from torch.optim.lr_scheduler import ExponentialLR

AggSmapeL=[]
AggWapeL=[]
AggPeL=[]
times_org=[]
TimesL=[]
wape_agg_lstm=np.zeros(50, dtype=np.float64)
df_lstm=np.zeros((30,50), dtype=np.float64)

In [19]:
AggSmapeL=[0]*50
AggWapeL=[0]*50
AggPeL=[0]*50
times_org=[0]*50
TimesL=[0]*50

In [20]:
obs_grp=[39,38,30,20,33]

In [21]:
# for i in range(49,50):
#     print('cluster id: ',i+1)
#     current_date = '20220831'
#     test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
#     #for top cluster:
#     #path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
#     path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
#     # read in data
#     df_panda = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
#     df_panda=df_panda[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
#     #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
#     Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
#     for cols in Cols:  
#         df_panda[cols]=df_panda[cols].astype(float,errors='raise')

#df_panda.to_csv('df_20_noF.csv', index=False)



In [22]:
seed=43
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [23]:
month=0
print('Running for the month of :',name[month])
################LSTM##########################
################LSTM##########################
for i in range(50):
    print('cluster id: ',i+1)
    current_date = '20220831'
    test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
    #for top cluster:
    path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
    #path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
    # read in data
    df_panda = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
    df_panda=df_panda[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
    #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
    Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
    for cols in Cols:  
        df_panda[cols]=df_panda[cols].astype(float,errors='raise')
    start=time.time()
    start=time.time()
    minError=1e10
    param = {
        "learning_rate": [.1,.01,.001],
        "hidden_size": [10,20],
        'batch_size': [32],
        'dropout': [0.05,.1],
        'gamma': [0.9],
        'optimizer_name': ['AdamW'],
        'num_layer': [1,2,3],
        'relu': [False]
    }
    prep = dataprep(df_panda, input_window=90, output_window=30, stride=1,start=start_month[month])
    print('Beginning the training...')
    train_set_in, train_set_out, val_set_in, val_set_out, test_in_set, test_out_set,df_test_org,  q,wq = prep._get_data()
    #print('print len of q, value and wq value',len(q),q[0],wq[0])
    train_set_in = torch.from_numpy(train_set_in.astype(np.float64)).float()
    train_set_out = torch.from_numpy(train_set_out.astype(np.float64)).float()
    val_set_in = torch.from_numpy(val_set_in.astype(np.float64)).float()
    val_set_out = torch.from_numpy(val_set_out.astype(np.float64)).float()
    test_in_set = torch.from_numpy(test_in_set.astype(np.float64)).float()
    test_out_set = torch.from_numpy(test_out_set.astype(np.float64)).float()
    print('val shape is: ',val_set_in.shape, val_set_out.shape)
    end1=time.time()
    if len(train_set_in.shape) < 3:
        train_set_in = train_set_in.reshape(train_set_in.shape[0], train_set_in.shape[1], 1)
        val_set_in = val_set_in.reshape(val_set_in.shape[0], val_set_in.shape[1], 1)
        test_in_set = test_in_set.reshape(test_in_set.shape[0], test_in_set.shape[1], 1)
        test_out_set=test_out_set.reshape(test_out_set.shape[0], test_out_set.shape[1])
    counter=1
    for lr in param['learning_rate']:
        for h_s in param['hidden_size']:
            for b_s in param['batch_size']:
                for dp in param['dropout']:
                    for opt in param['optimizer_name']:
                        for n_l in param['num_layer']:
                            for act in param['relu']:
                                for gm in param['gamma']:
                                    config = {
                                        "num_epochs": 200,
                                        "learning_rate": lr,
                                        "hidden_size": h_s,
                                        "window_size": 30,
                                        "output_size": 30,
                                        'batch_size': b_s,
                                        'dropout': dp,
                                        'dropout1': 0,
                                        'gamma': gm,
                                        'optimizer_name': opt,
                                        'num_layer': n_l,
                                        'relu': act
                                    }
                                    #torch.manual_seed(seed)
                                    print('trial run: ',counter)
                                    counter+=1
                                    e1,e2,e3 = running_lstm(config, save_model=True)
                                    if (e1+e2+e3)/3<minError:
                                        minError=(e1+e2+e3)/3
                                        best_config=config
                                        print('the  smape is {} wape is {} and pe is {} and total is {}'.format(e1,e2,e3,(e1+e2+e3)/3))
                                        print('the config is {}',config)
    if minError>=.7:
        param = {
        "learning_rate": [.005,.1,.0001],
        "hidden_size": [15,30],
        'batch_size': [64],
        'dropout': [0.3,.5],
        'gamma': [0.9],
        'optimizer_name': ['Adam','AdamW'],
        'num_layer': [3,5],
        'relu': [True]}
        for lr in param['learning_rate']:
            for h_s in param['hidden_size']:
                for b_s in param['batch_size']:
                    for dp in param['dropout']:
                        for opt in param['optimizer_name']:
                            for n_l in param['num_layer']:
                                for act in param['relu']:
                                    for gm in param['gamma']:
                                        config = {
                                            "num_epochs": 200,
                                            "learning_rate": lr,
                                            "hidden_size": h_s,
                                            "window_size": 30,
                                            "output_size": 30,
                                            'batch_size': b_s,
                                            'dropout': dp,
                                            'dropout1': 0,
                                            'gamma': gm,
                                            'optimizer_name': opt,
                                            'num_layer': n_l,
                                            'relu': act
                                        }
                                        print('trial run: ',counter)
                                        counter+=1
                                        e1,e2,e3 = running_lstm(config, save_model=True)
                                        
                                        #print('the config is {}',config)
                                        if (e1+e2+e3)/3<minError:
                                            minError=(e1+e2+e3)/3
                                            best_config=config
                                            print('the  smape is {} wape is {} and pe is {} and total is {}'.format(e1,e2,e3,(e1+e2+e3)/3))
                                            print('the config is {}',config)

    end = time.time()
    print('thus best config is', best_config)
    Wsmape, Wape, Wpe, smape_dis, wape_dis, pe_dis, smape_agg, wape_agg, pe_agg, model, df_lstm[:,i],wape_agg_lstm[i] = train_lstm(best_config, save_model=True)
    print('val wape is: ',wape_agg_lstm[i])
    print('total time ', end1 - end)
    print('test Wsmape: ', Wsmape, ' test Wape: ', Wape, ' test Wpe: ', Wpe)
    print('Smape dis',smape_dis,'Wape dis',wape_dis,'PE dis',pe_dis)
    print('Agg SMAPE: ',smape_agg,'Agg WAPE: ',wape_agg,'Agg PE: ',pe_agg)
    print('total time ', end1 - end)

    #print('time needed to run the model: ',end-start)
    AggSmapeL.append(smape_agg)
    AggWapeL.append(wape_agg)
    AggPeL.append(pe_agg)
    SMAPEsL.append(Wsmape)
    WAPEsL.append(Wape)
    PEsL.append(Wpe)
    median_smape.append(smape_dis[0])
    Eighty_per_smape.append(smape_dis[1])
    Ninty_per_smape.append(smape_dis[2])
    median_wape.append(wape_dis[0])
    Eighty_per_wape.append(wape_dis[1])
    Ninty_per_wape.append(wape_dis[2])
    median_pe.append(pe_dis[0])
    Eighty_per_pe.append(pe_dis[1])
    Ninty_per_pe.append(pe_dis[2])
    TimesL.append(end1-end)
    times_org.append(end1-start)
    print('===============================================')
    
    

Running for the month of : September
cluster id:  1


AnalysisException: 'Path does not exist: gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample_1005862_1007220_1044020_3392907_tft_data_20220831.csv;'

In [ ]:
#df_panda.to_csv('df_36_noF.csv', index=False)

In [ ]:
print('TimesL=',TimesL)
print('AggSmapeL=',AggSmapeL)
print('AggWapeL=',AggWapeL)
print('AggPeL=',AggPeL)
print('times_org=',times_org)

In [ ]:
print('Running for the month of :',name[month])

In [ ]:
#df_panda.to_csv('df_36.csv', index=False)

In [ ]:
class statModels:
    def __init__(self):
        # self.df_train=df
        self.models = {}
        self.sc = MinMaxScaler(feature_range=(0, 1))
        self.clus_wape={}
        self.clus_smape={}
        self.clus_pe={}
        self.wape_Weighted={}
        self.stores_times={}
        self.dfStore=None
    def arimamodel(self, timeseriesarray):
        autoarima_model = pmd.auto_arima(timeseriesarray, start_p=0, d=0, trace=1,
                                         suppress_warnings=True, seasonal=True, m=12, D=1,
                                         start_q=0, error_action='ignore', stationary=True, njob=-1,
                                         test="adf", stepwise=True)
        return autoarima_model
    def preprocessing(self, df_train):
        temp = df_train  # np.log(df_train)
        temp = temp.reshape((-1, 1))
        return self.sc.fit_transform(temp)

    def postprocess(self, prediction, df_test_org,dfx_test, q, wq, ensemble=False):
        # prediction=np.exp(prediction)
        if type(prediction)==pd.core.frame.DataFrame:
            prediction = prediction.values.reshape(-1, 1)
        else:prediction = prediction.reshape(-1, 1)
        if not ensemble:
            prediction = self.sc.inverse_transform(prediction)
        re_elements = wq.shape[0]  # testY.shape[-1]
        testing_predict = prediction.reshape(-1, )
        #else:testing_predict = prediction
        clus_wape,clus_smape,clus_pe=WAPE(dfx_test,testing_predict),SMAPE(dfx_test,testing_predict),PE(dfx_test,testing_predict)
        
        testing_predict = np.tile(testing_predict, (re_elements, 1))
        testing_predict = testing_predict.T     
        testing_predict *= q

#         Wsmape, Wwape,Wpe = estimate(df_test_org.values, testing_predict, wq)
#         smape_dis,wape_dis,pe_dis=metric_dist(df_test_org.values, testing_predict, wq,metric='smape'),metric_dist(df_test_org.values, testing_predict, wq,metric='wape') ,metric_dist(df_test_org.values, testing_predict, wq,metric='pe')
        return clus_smape,clus_wape,clus_pe, prediction  # ,testing_predict

    def training_infer(self, df_train, df_test, df_test_org,q, wq, forecast_period=30,lstm_pred=None):
        '''
        df_test: pandas (shape:(:,1))
        df_test_org: pandas (shape(:,no_ts))
        '''

        # self.df_train_log,self.df_test_log=test_train_spl(self.df,ratio=.8)
        ###
        fh = np.arange(forecast_period) + 1
        ###
        df_train['daily_supply'] = self.preprocessing(df_train.values)
        self.df_train = df_train
        # print(df_train['daily_supply'])
        df_store = df_test.to_frame()
        df_store = df_store.rename(columns={0: 'daily_supply'})
        ###################################################
        #df_store['lstm']=lstm_pred
        #print('naive')
        # naive
        start = time.time()
        forecaster=NaiveForecaster(strategy="last")
        forecaster.fit(self.df_train)
        y_pred = forecaster.predict(fh)
        df_store['naive'] = y_pred.values
        # print(df_store['snaive'])
        # print(df_store)
        clus_smape,clus_wape,clus_pe,df_store['naive'] = self.postprocess(df_store['naive'].values, df_test_org,df_test,q, wq)
        
        end = time.time()
        self.clus_smape['naive']=clus_smape
        self.clus_wape['naive']=clus_wape
        self.clus_pe['naive']=clus_pe
        self.stores_times['naive'] = end - start
        self.models['naive'] = forecaster
        

        #print('snaive')
        # snaive
        start = time.time()
#         y = self.pysnaive(self.df_train, 52, forecast_period)[1].iloc[:forecast_period]
#         df_store['snaive'] = y.values
        forecaster1=NaiveForecaster(strategy="last",sp=12)
        forecaster1.fit(self.df_train)
        y_pred = forecaster1.predict(fh)
        clus_smape,clus_wape,clus_pe,df_store['snaive'] = self.postprocess(y_pred.values, df_test_org,df_test,q, wq)
        
        end = time.time()
        self.clus_smape['snaive']=clus_smape
        self.clus_wape['snaive']=clus_wape
        self.clus_pe['snaive']=clus_pe
        
        self.stores_times['snaive'] = end - start
        self.models['snaive'] = forecaster1
        
        #print('arima')
        start = time.time()
        try: 
            model_fit = ARIMA(self.df_train, order=(1, 1, 1)).fit()
            df_store['arima'], _, _ = model_fit.forecast(forecast_period)  # 95% conf
            print('done')
        except (np.linalg.linalg.LinAlgError,ValueError):
            model_fit = self.arimamodel(self.df_train)
            df_store['arima']=model_fit.predict(forecast_period)
            print('done3')
#         else:
#             model_fit = ARIMA(self.df_train, order=(0, 0, 0)).fit()
#             df_store['arima']=model_fit.predict(forecast_period)
#             print('done3')
            
        
        #df_store['arima']= model_fit.forecast(forecast_period).values  # 95% conf
        clus_smape,clus_wape,clus_pe,df_store['arima'] = self.postprocess(df_store['arima'].values, df_test_org,df_test,q, wq)
        
        end = time.time()

        self.models['arima'] = model_fit
        # self.stores_mape['arima'] = mape
        self.clus_smape['arima']=clus_smape
        self.clus_wape['arima']=clus_wape
        self.clus_pe['arima']=clus_pe
        
        self.stores_times['arima'] = end - start
        self.models['arima'] = model_fit

        #print('ses')
        # ses
        start = time.time()
        fit3 = SimpleExpSmoothing(self.df_train, initialization_method="estimated").fit()
        df_store['ses'] = fit3.forecast(forecast_period).values
        clus_smape,clus_wape,clus_pe,df_store['ses'] = self.postprocess(df_store['ses'].values, df_test_org,df_test,q, wq)
        
        end = time.time()
        self.models['ses'] = fit3

        #self.stores_mape['ses'] = mape
        self.clus_smape['ses']=clus_smape
        self.clus_wape['ses']=clus_wape
        self.clus_pe['ses']=clus_pe
        self.stores_times['ses'] = end - start
        #print('holts')
        # holts
        start = time.time()
        try:
            fit3 = Holt(self.df_train, damped_trend=True, exponential=True, initialization_method="estimated").fit(
                smoothing_level=0.8, smoothing_trend=0.1)
        except (np.linalg.linalg.LinAlgError,ValueError):
            fit3 = Holt(self.df_train, damped_trend=True, exponential=False, initialization_method="estimated").fit(
                smoothing_level=0.8, smoothing_trend=0.1)
        df_store['holts'] = fit3.forecast(forecast_period).values
        clus_smape,clus_wape,clus_pe,df_store['holts'] = self.postprocess(df_store['holts'].values, df_test_org,df_test,q, wq)
        
        end = time.time()

        self.models['holts'] = fit3
        #self.stores_mape['holts'] = mape
        self.clus_smape['holts']=clus_smape
        self.clus_wape['holts']=clus_wape
        self.clus_pe['holts']=clus_pe
        self.stores_times['holts'] = end - start
        #print('Exp_smoothing')
        # Exp_smoothing
        start = time.time()
        try:
            fit4 = ExponentialSmoothing(self.df_train, seasonal_periods=52, trend="add", seasonal="add",
                                        damped_trend=True, use_boxcox=True, initialization_method="estimated", ).fit()
        except:
            fit4 = ExponentialSmoothing(self.df_train, seasonal_periods=52, trend="add", seasonal="add",
                                        damped_trend=True, use_boxcox=False, initialization_method="estimated", ).fit()
        df_store['Es'] = fit4.forecast(forecast_period).values
        clus_smape,clus_wape,clus_pe,df_store['Es'] = self.postprocess(df_store['Es'].values, df_test_org,df_test,q, wq)
        
        end = time.time()
        self.models['Es'] = fit4
        #self.stores_mape['Es'] = mape
        self.clus_smape['Es']=clus_smape
        self.clus_wape['Es']=clus_wape
        self.clus_pe['Es']=clus_pe
        self.stores_times['Es'] = end - start

        
        #print('Auto_ETS')
        # Auto_ETS
        start = time.time()
        try: 
            forecaster = AutoETS(auto=True, sp=12, n_jobs=-1, error='add', trend='True', damped_trend='True', seasonal='add')
            forecaster.fit(self.df_train)
        except (ValueError):
            forecaster = AutoETS(auto=False, sp=7, n_jobs=-1, error='add')
            forecaster.fit(self.df_train)
        df_store['auto_ets'] = forecaster.predict(fh).values
        clus_smape,clus_wape,clus_pe,df_store['auto_ets'] = self.postprocess(df_store['auto_ets'].values, df_test_org,df_test,q, wq)
        
        end = time.time()
        self.models['auto_ets'] = forecaster
        #self.stores_mape['auto_ets'] = mape
        self.clus_smape['auto_ets']=clus_smape
        self.clus_wape['auto_ets']=clus_wape
        self.clus_pe['auto_ets']=clus_pe
        self.stores_times['auto_ets'] = end - start
        
        #theta
        start = time.time()
        forecaster4 = ThetaForecaster(sp=52, deseasonalize=False)
        forecaster4.fit(self.df_train)
        df_store['theta'] = forecaster4.predict(fh).values
        clus_smape,clus_wape,clus_pe,df_store['theta'] = self.postprocess(df_store['theta'].values, df_test_org,df_test,q, wq)
        
        end = time.time()

        self.models['theta'] = forecaster4
        #self.stores_mape['theta'] = mape
        self.clus_smape['theta']=clus_smape
        self.clus_wape['theta']=clus_wape
        self.clus_pe['theta']=clus_pe

        self.stores_times['theta'] = end - start

        #print('Croston')
        # Croston
        start = time.time()
        forecaster5 = Croston(smoothing=.2)
        forecaster5.fit(self.df_train)
        df_store['croston'] = forecaster5.predict(fh).values
        clus_smape,clus_wape,clus_pe,df_store['croston'] = self.postprocess(df_store['croston'].values, df_test_org,df_test,q, wq)
        
        end = time.time()

        self.models['croston'] = forecaster5
        #self.stores_mape['croston'] = mape
        self.clus_smape['croston']=clus_smape
        self.clus_wape['croston']=clus_wape
        self.clus_pe['croston']=clus_pe
        self.stores_times['croston'] = end - start

        #prophet
        start = time. time()
        df_train=self.df_train.squeeze()
        df_train.index.name='Period'
        df_train.index=df_train.index.to_timestamp()
        forecaster6 = Prophet(seasonality_mode='additive',n_changepoints=int(len(df_train) / 12),
            add_country_holidays={'country_name': 'US'},yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
        forecaster6.fit(df_train)
        df_store['prophet'] = forecaster6.predict(fh).values
        clus_smape,clus_wape,clus_pe,df_store['prophet'] = self.postprocess(df_store['prophet'].values, df_test_org,df_test,q, wq)
        
        end = time.time()

        self.models['prophet'] = forecaster6
        # self.stores_mape['croston'] = mape
        self.clus_smape['prophet']=clus_smape
        self.clus_wape['prophet']=clus_wape
        self.clus_pe['prophet']=clus_pe
        self.stores_times['prophet'] = end - start

        #print('Average')
        # Average of stats models
        tog_time = np.sum(list(self.stores_times.values()))
        start = time.time()
        df_temp = df_store.drop(['daily_supply'], axis=1)  # df_store.loc[:, df_store.columns != 'daily_supply']
        df_store['Avg_ensemble'] = df_temp.mean(axis=1).values

        clus_smape,clus_wape,clus_pe,df_store['Avg_ensemble'] = self.postprocess(df_store['Avg_ensemble'].values, df_test_org,df_test,q, wq,ensemble=True)
        
        end = time.time()
        #self.stores_mape['Avg_ensemble'] = mape
        self.clus_smape['Avg_ensemble']=clus_smape
        self.clus_wape['Avg_ensemble']=clus_wape
        self.clus_pe['Avg_ensemble']=clus_pe
        self.stores_times['Avg_ensemble'] = (end - start) + tog_time-self.stores_times['Lstm']#%%%%%%%%%%%%
        self.dfStore=df_store

    def weighted_avg(self,df_train, df_test,df_val_org, df_test_org,q, wq, forecast_period=60,lstm_pred=None):
        
        fh = np.arange(forecast_period) + 1
        ###
        df_train['daily_supply'] = self.preprocessing(df_train.values)
        self.df_trainW = df_train
        # print(df_train['daily_supply'])
        df_store = df_test.to_frame()
        df_store = df_store.rename(columns={0: 'daily_supply'})
        #############################################
        start = time.time()
        forecaster=NaiveForecaster(strategy="last")
        forecaster.fit(self.df_trainW)
        y_pred = forecaster.predict(fh)

        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['naive'] = (smape+wape+pe)/3

        

        #print('snaive')
        # snaive
       # start = time.time()
#         y = self.pysnaive(self.df_train, 52, forecast_period)[1].iloc[:forecast_period]
#         df_store['snaive'] = y.values
        forecaster1=NaiveForecaster(strategy="last",sp=12)
        forecaster1.fit(self.df_trainW)
        y_pred = forecaster1.predict(fh)
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['snaive'] = (smape+wape+pe)/3
        
        #print('arima')
        #start = time.time()
        try: 
            model_fit = ARIMA(self.df_trainW, order=(1, 1, 1)).fit()
            y_pred, _, _ = model_fit.forecast(forecast_period)  # 95% conf
        except (np.linalg.linalg.LinAlgError,ValueError):
            model_fit = self.arimamodel(self.df_trainW)
            y_pred=model_fit.predict(forecast_period)
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['arima'] = (smape+wape+pe)/3
    

        fit3 = SimpleExpSmoothing(self.df_trainW, initialization_method="estimated").fit()
        y_pred = fit3.forecast(forecast_period).values
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['ses'] = (smape+wape+pe)/3
        
        #print('holts')
        # holts
        start = time.time()
        try:
            fit3 = Holt(self.df_trainW, damped_trend=True, exponential=True, initialization_method="estimated").fit(
                smoothing_level=0.8, smoothing_trend=0.1)
        except (np.linalg.linalg.LinAlgError,ValueError):
            fit3 = Holt(self.df_trainW, damped_trend=True, exponential=False, initialization_method="estimated").fit(
                smoothing_level=0.8, smoothing_trend=0.1)
        y_pred = fit3.forecast(forecast_period).values
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['holts'] = (smape+wape+pe)/3
        # Exp_smoothing
        #start = time.time()
        try:
            fit4 = ExponentialSmoothing(self.df_trainW, seasonal_periods=52, trend="add", seasonal="add",
                                        damped_trend=True, use_boxcox=True, initialization_method="estimated", ).fit()
        except:
            fit4 = ExponentialSmoothing(self.df_trainW, seasonal_periods=52, trend="add", seasonal="add",
                                        damped_trend=True, use_boxcox=False, initialization_method="estimated", ).fit()
        y_pred = fit4.forecast(forecast_period).values
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['Es'] = (smape+wape+pe)/3

        
        #print('Auto_ETS')
        # Auto_ETS
        #start = time.time()
        try:
            forecaster = AutoETS(auto=True, sp=12, n_jobs=-1, error='add', trend='True', damped_trend='True', seasonal='add')
            forecaster.fit(self.df_trainW)
        except (ValueError):
            forecaster = AutoETS(auto=False, sp=7, n_jobs=-1, error='add')
            forecaster.fit(self.df_trainW)
        y_pred = forecaster.predict(fh).values
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['auto_ets'] = (smape+wape+pe)/3
        
        #theta
        #start = time.time()
        forecaster4 = ThetaForecaster(sp=52, deseasonalize=False)
        forecaster4.fit(self.df_trainW)
        y_pred = forecaster4.predict(fh).values
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['theta'] = (smape+wape+pe)/3

        #print('Croston')
        # Croston
        #start = time.time()
        forecaster5 = Croston(smoothing=.2)
        forecaster5.fit(self.df_trainW)
        y_pred = forecaster5.predict(fh).values
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['croston'] = (smape+wape+pe)/3

        #prophet
        #start = time. time()
        df_train=self.df_trainW.squeeze()
        df_train.index.name='Period'
        df_train.index=df_train.index.to_timestamp()
        forecaster6 = Prophet(seasonality_mode='additive',n_changepoints=int(len(df_train) / 12),
            add_country_holidays={'country_name': 'US'},yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
        forecaster6.fit(df_train)
        y_pred = forecaster6.predict(fh).values
        
        smape,wape,pe,_= self.postprocess(y_pred[:30], df_val_org,df_val_org,q, wq)
        self.wape_Weighted['prophet'] = (smape+wape+pe)/3
        
        store_error = self.wape_Weighted.copy()

        for k, v in self.wape_Weighted.items():
            store_error[k] = 1 / v
        norms = list(store_error.values())
        norm = 0
        for i in norms: norm += i
        for k, v in store_error.items():
            store_error[k] = v / norm
        self.store_error = store_error
        #print('weighted')
        # weighted average
        #del self.dfStore['daily_supply']
        print(self.store_error)
        subset = self.dfStore.drop(['daily_supply', 'Avg_ensemble'], axis=1)#.columns
        self.dfStore['Weighted_ensemble'] = (subset * store_error).sum(1).values
        clus_smape,clus_wape,clus_pe,self.dfStore['Weighted_ensemble'] = self.postprocess(self.dfStore['Weighted_ensemble'].values, df_test_org,df_test,q, wq,ensemble=True)
        
        end = time.time()
        self.clus_smape['Weighted_ensemble']=clus_smape
        self.clus_wape['Weighted_ensemble']=clus_wape
        self.clus_pe['Weighted_ensemble']=clus_pe
        self.stores_times['Weighted_ensemble'] = (end - start)#+self.stores_times['Lstm']
        #self.dfStore['Weighted_ensemble']=df_store['Weighted_ensemble'].values
    


In [ ]:
def _generate_train_test_data(df, window_size=30,val_req=False,start='2020-11-01'):
    start = pd.to_datetime(start, format='%Y-%m-%d') #+ relativedelta(months=months)
    end = start + relativedelta(days=365 + 30) #+ relativedelta(months=months)    
    df3 = df.astype({'query_idx': 'string'})
    df3.sort_values('ds', inplace=True)
    df1 = pd.pivot_table(df3, values='daily_supply', index=['ds'],
                         columns=['group_id', 'query_idx', 'SearchKeyValue', 'LocationKeyValue'])

    df1.columns = ['_'.join(col) for col in df1.columns.values]
    print('Total TS before filter: ', df1.shape[1])

    df1.index = pd.to_datetime(df1.index)

#     start = df1.index[0]#****
#     end = df1.index[-1]#*****
            
    
    #start,end=pd.to_datetime('2020-05-02',format='%Y-%m-%d'),pd.to_datetime('2022-05-02',format='%Y-%m-%d')
#     start1=df1.index.sort_values()[0]
#     end1=df1.index.sort_values()[len(df1)-1]
#     #print(df1.index.sort_values())
#     if end1<end:end=end1
#     if start1>start:start=start1
    df1=df1[(df1.index>=start) & (df1.index<=end)]
    df1.sort_index(inplace=True)

    train_test_split = end - pd.to_timedelta(30, unit='d')  ############30 needs to be a variable
    # start_temp2 = pd.to_datetime((train_test_split + relativedelta(days=-4 * window_size)).strftime('%Y-%m-%d'))

    end_train = pd.to_datetime((train_test_split).strftime('%Y-%m-%d'))
    start_test = pd.to_datetime((train_test_split + relativedelta(days=1)).strftime('%Y-%m-%d'))
    #print(df1.index.unique())
    dates = pd.date_range(start=start, end=end, freq='D').format()
    s = pd.Series(np.nan, index=pd.date_range(start, end, freq='D'))
    #df1['ds']=pd.to_datetime(df_['ds'])
    #df_.set_index('ds',inplace=True)
    df1=pd.concat([df1,s[~s.index.isin(df1.index)]]).sort_index()
    
    #print(df1)
    #del df1['0']
    #dates = pd.date_range(start=start, end=end, freq='D').format()
    #df1=df1.reindex(dates, fill_value=0)
    df1.index = pd.to_datetime(df1.index)
    df1 = df1.fillna(0)
#########################    
    #df1 = df1.loc[:, df1.isin([0]).sum(axis=0) <= 50]
########################
    #     df1=df1.loc[:,df1.notna().sum()>90]
    #     df1 = df1.fillna(0)
    del df1[0]
    # impute_missing_time_modify(temp, train_test_split, end, 4, train=True)
    df_train, df_test = df1[(df1.index <= end_train)], df1[df1.index >= start_test]
    # temp_train, temp_test = temp[temp['ds'] < train_test_split], temp[temp['ds'] >= train_test_split]
    print('Total TS after filter: ', df1.shape[1])
    df_train.sort_index(inplace=True)
    #print(df_train.iloc[-14:,:])
    
    #df_train_temp=df_train_temp[(df_train_temp.index <= end_train)], df_train_temp[df_train_temp.index >= start_test]
    
    qtrain = df_train.iloc[-14:,:].sum(axis=0) / np.sum(df_train.iloc[-14:,:].sum(axis=0))
    qtest = df_test.sum(axis=0) / np.sum(df_test.sum(axis=0))  # df_train.iloc[-30:,].sum(axis=0)/np.sum(df_train.iloc[-30:,].sum(axis=0))*100
    # average=df_train.mean(axis=0).values
    # df_train+=average
    # df_test+= average
    train_date, test_date = df_train.index, df_test.index
    df_train.reset_index(inplace=True, drop=True)
    df_test.reset_index(inplace=True, drop=True)
    # df.values.sum()
    # q=df.sum(axis=1)/np.sum(np.sum(df))
    # print(q)
    if val_req:
        df_train,df_val=df_train.iloc[:-30,:],df_train.iloc[-30:,:]
        return df_train, df_val,df_test, train_date.values[:-30], test_date.values, qtrain, qtest
    return df_train, df_test, train_date.values, test_date.values, qtrain, qtest


In [ ]:
Times=[]
AggSmape=[]
AggWape=[]
AggPe=[]


In [ ]:
# month=0
# start_month[month]

In [ ]:

counter=0
for i in range(50):
    if i not in [44,47,48]:
        st=time.time()
        print('cluster id: ',i+1)
        current_date = '20220831'
        test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
        print('group id',test_group_id)
        #for top cluster:
        #path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
        path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
        # read in data
        df = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
        df=df[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
        #print('query id: ',df['query_idx'].unique())
        #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
        Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
        for cols in Cols:df[cols]=df[cols].astype(float,errors='raise')
        #print('time needed to preprocess the data is: ',time.time()-st)
        df_train,df_test,train_date,test_date,q,wq=_generate_train_test_data(df,start=start_month[month])
        df_trainW,df_valW,_,train_dateW,_,_,_=_generate_train_test_data(df,val_req=True,start=start_month[month])
        df1_train,df1_test=df_train.sum(axis=1),df_test.sum(axis=1)
        df1_trainW,df_valW=df_trainW.sum(axis=1),df_valW.sum(axis=1)
        df1_train,df1_trainW,df_valW=df1_train.to_frame(),df1_trainW.to_frame(),df_valW.to_frame()
        df1_train.index=train_date
        df1_trainW.index=train_dateW
        df1_train.index = df1_train.index.to_period(freq = 'D')
        df1_trainW.index = df1_trainW.index.to_period(freq = 'D')
        #df_test_log_temp.index = df_test_log_temp.index.to_period(freq = 'D')
        df1_train=df1_train.rename(columns={0:'daily_supply'})
        df1_trainW=df1_trainW.rename(columns={0:'daily_supply'})
    #     if i==0:
    #         print(q,wq)
        start=time.time()
        Models=statModels()

        Models.clus_smape['Lstm']=AggSmapeL[counter]
        Models.clus_wape['Lstm']=AggWapeL[counter]
        Models.clus_pe['Lstm']=AggPeL[counter]
        #Models.models['Lstm'] = fit4
        #self.stores_mape['Es'] = mape
        Models.stores_times['Lstm'] = TimesL[counter]
        ##################
        #Models.wape_Weighted['Lstm'] = wape_agg_lstm[i]

        Models.training_infer(df1_train,df1_test,df_test,q.values,wq.values,df1_test.shape[0],df_lstm[:,i])#no_ts
                             #(df_train, df_test, df_test_org,q, wq, forecast_period=30,lstm_pred=None)
        Models.weighted_avg(df1_trainW, df1_test,df_valW.values, df_test,q.values,wq.values, forecast_period=60,lstm_pred=df_lstm[:,i])

        end=time.time()
        print('Wape is: ',Models.clus_wape)
        print('Smape is: ',Models.clus_smape)
        print('Pe is: ',Models.clus_smape)
        print('Time is:' ,Models.clus_pe)
        #print('time needed to run the model: ',end-start)
        Times.append(Models.stores_times)
        AggSmape.append(Models.clus_smape)
        AggWape.append(Models.clus_wape)
        AggPe.append(Models.clus_pe)
        #training_j.append(i)
        counter+=1
    print('===============================================')
    

In [ ]:
print(counter,len(AggPe))

In [ ]:
# Ls=[AggWape[i]['Avg_ensemble'] for i in range(len(AggWape))]
# res=[]
# for i in range(6):
#     res.append(np.argmax(Ls))
#     print(np.argmax(Ls),':',Ls[np.argmax(Ls)])
#     Ls[np.argmax(Ls)]=0
# Ls=[AggWape[i]['Avg_ensemble'] for i in range(len(AggWape))]
# for i in range(3):
#     res.append(np.argmin(Ls))
#     print(np.argmin(Ls),':',Ls[np.argmin(Ls)])
#     Ls[np.argmin(Ls)]=1

# res

In [ ]:
#res[4]=45

In [ ]:
# fig, axes = plt.subplots(nrows=3, ncols=3)
# sh=[[i,j] for i in range(3) for j in range(3)]
# counter=0
# matplotlib.rcParams['figure.figsize'] = (30, 20)

# for i in res:
#     if i not in [44,47,48]:
#         st=time.time()
#         print('cluster id: ',i+1)
#         current_date = '20220831'
#         test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
#         print('group id',test_group_id)
#         #for top cluster:
#         #path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
#         path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
#         # read in data
#         df = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
#         df=df[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
#         #print('query id: ',df['query_idx'].unique())
#         #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
#         Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
#         for cols in Cols:df[cols]=df[cols].astype(float,errors='raise')
#         #print('time needed to preprocess the data is: ',time.time()-st)
#         df_train,df_test,train_date,test_date,q,wq=_generate_train_test_data(df,start=start_month[month])
#         df_trainW,df_valW,_,train_dateW,_,_,_=_generate_train_test_data(df,val_req=True,start=start_month[month])
#         df1_train,df1_test=df_train.sum(axis=1),df_test.sum(axis=1)
#         df1_trainW,df_valW=df_trainW.sum(axis=1),df_valW.sum(axis=1)
#         df1_train,df1_trainW,df_valW=df1_train.to_frame(),df1_trainW.to_frame(),df_valW.to_frame()
#         df1_train.index=train_date
#         df1_trainW.index=train_dateW
        
#         df1_train.index = df1_train.index.to_period(freq = 'D')
#         df1_trainW.index = df1_trainW.index.to_period(freq = 'D')
#         #df_test_log_temp.index = df_test_log_temp.index.to_period(freq = 'D')
#         df1_train=df1_train.rename(columns={0:'daily_supply'})
#         df1_trainW=df1_trainW.rename(columns={0:'daily_supply'})
#         start=time.time()
#         df_train=df1_train.copy()
#         Models=statModels()
        
#         Models.clus_smape['Lstm']=0
#         Models.clus_wape['Lstm']=0
#         Models.clus_pe['Lstm']=0
#         #Models.models['Lstm'] = fit4
#         #self.stores_mape['Es'] = mape
#         Models.stores_times['Lstm'] = 0
        
#         Models.training_infer(df1_train,df1_test,df_test,q.values,wq.values,df1_test.shape[0],df_lstm[:,i])#no_ts
#                              #(df_train, df_test, df_test_org,q, wq, forecast_period=30,lstm_pred=None)
#         Models.weighted_avg(df1_trainW, df1_test,df_valW.values, df_test,q.values,wq.values, forecast_period=60,lstm_pred=df_lstm[:,i])

#         end=time.time()
#         print('Wape is: ',Models.clus_wape)
#         print('Smape is: ',Models.clus_smape)
#         print('Pe is: ',Models.clus_smape)
#         print('Time is:' ,Models.clus_pe)
#         df1_test=df1_test.to_frame()
#         df1_test.index=test_date
#         df1_test=df1_test.rename(columns={0:'daily_supply_test'})
#         df_train=df_train.rename(columns={'daily_supply':'daily_supply_train'})
#         #print('time needed to run the model: ',end-start)
#         Times.append(Models.stores_times)
#         AggSmape.append(Models.clus_smape)
#         AggWape.append(Models.clus_wape)
#         AggPe.append(Models.clus_pe)
#         pred=Models.dfStore['Avg_ensemble']
#         pred.index=test_date
#         pred=pred.to_frame()
#         pred=pred.rename(columns={'Avg_ensemble':'daily_supply_pred'})

#         df_train.plot(title='cluster id: '+str(i)+ ' with Agg WAPE: '+str(Models.clus_wape['Avg_ensemble']),ax=axes[sh[counter][0],sh[counter][1]])
#         df1_test.plot(ax=axes[sh[counter][0],sh[counter][1]])
#         pred.plot(ax=axes[sh[counter][0],sh[counter][1]])
#         counter+=1

In [ ]:

Time={i:[] for i in Times[0]}

AggSmapes={i:[] for i in AggSmape[0]}
AggWapes={i:[] for i in AggWape[0]}
AggPes={i:[] for i in AggPe[0]}


In [ ]:

for i in range(counter):

    for keys,values in Times[i].items():
        Time[keys].append(values)
    for keys,values in AggSmape[i].items():
        AggSmapes[keys].append(values)
    for keys,values in AggWape[i].items():
        AggWapes[keys].append(values)
    for keys,values in AggPe[i].items():
        AggPes[keys].append(values)
    
    

In [ ]:

print('Time is: ',Time)
print('====================')
print('AggSmapes is: ',AggSmapes)
print('====================')
print('AggWapes is: ',AggWapes)
print('====================')
print('AggPes is: ',AggPes)
print('====================')

print('====================')
print('Original LSTM time is', times_org)

In [ ]:
len(AggPes["Lstm"])

In [ ]:
%matplotlib inline
import numpy as np
avg_,med_,e80_,e90_,name=[],[],[],[],[]
#a=[MAPE_clus,MAPE_agg,WAPE_clus,WAPE_agg]
for k,v in Time.items():
    avg_.append(np.mean(v))
    med_.append(np.percentile(v,50))
    e80_.append(np.percentile(v,80))
    e90_.append(np.percentile(v,90))
    name.append(k)



x_axis = np.arange(len(Time))

# Multi bar Chart

bar_w=.1
plt.bar(x_axis -0.15, avg_, width=bar_w, label = 'Avg',color='red')
plt.bar(x_axis -0.05, med_, width=bar_w, label = 'Median',color='green')
plt.bar(x_axis +0.05, e80_, width=bar_w, label = '80-%ile',color='cyan')
plt.bar(x_axis +0.15, e90_, width=bar_w, label = '90-%tile',color='black')

plt.xticks(x_axis, Time.keys(),rotation = 70)
plt.legend(bbox_to_anchor=(0.41,0.65))
plt.ylabel('Time (seconds)')
plt.xlabel('Model')
plt.show()  

In [ ]:
import numpy as np
avg_,med_,e80_,e90_,name=[],[],[],[],[]
#a=[MAPE_clus,MAPE_agg,WAPE_clus,WAPE_agg]
for k,v in AggPes.items():
    avg_.append(np.mean(v))
    med_.append(np.percentile(v,50))
    e80_.append(np.percentile(v,80))
    e90_.append(np.percentile(v,90))
    name.append(k)



x_axis = np.arange(len(AggPes))

# Multi bar Chart
plt.bar(x_axis -0.15, avg_, width=bar_w, label = 'Avg',color='red')
plt.bar(x_axis -0.05, med_, width=bar_w, label = 'Median',color='green')
plt.bar(x_axis +0.05, e80_, width=bar_w, label = '80-%ile',color='cyan')
plt.bar(x_axis +0.15, e90_, width=bar_w, label = '90-%tile',color='black')

plt.xticks(x_axis, AggPes.keys(),rotation = 70)
plt.legend(bbox_to_anchor=(0.41,0.65))
plt.ylabel('Agg Weighted PE')
plt.xlabel('Model')
plt.show()

In [ ]:
import numpy as np
avg_,med_,e80_,e90_,name=[],[],[],[],[]
#a=[MAPE_clus,MAPE_agg,WAPE_clus,WAPE_agg]
for k,v in AggSmapes.items():
    avg_.append(np.mean(v))
    med_.append(np.percentile(v,50))
    e80_.append(np.percentile(v,80))
    e90_.append(np.percentile(v,90))
    name.append(k)



x_axis = np.arange(len(AggPes))

# Multi bar Chart
plt.bar(x_axis -0.15, avg_, width=bar_w, label = 'Avg',color='red')
plt.bar(x_axis -0.05, med_, width=bar_w, label = 'Median',color='green')
plt.bar(x_axis +0.05, e80_, width=bar_w, label = '80-%ile',color='cyan')
plt.bar(x_axis +0.15, e90_, width=bar_w, label = '90-%tile',color='black')

plt.xticks(x_axis, AggPes.keys(),rotation = 70)
plt.legend(bbox_to_anchor=(0.41,0.65))
plt.ylabel('Agg Weighted SMAPE')
plt.xlabel('Model')
plt.show()

In [ ]:
import numpy as np
avg_,med_,e80_,e90_,name=[],[],[],[],[]
#a=[MAPE_clus,MAPE_agg,WAPE_clus,WAPE_agg]
for k,v in AggWapes.items():
    avg_.append(np.mean(v))
    med_.append(np.percentile(v,50))
    e80_.append(np.percentile(v,80))
    e90_.append(np.percentile(v,90))
    name.append(k)



x_axis = np.arange(len(AggPes))

# Multi bar Chart
plt.bar(x_axis -0.15, avg_, width=bar_w, label = 'Avg',color='red')
plt.bar(x_axis -0.05, med_, width=bar_w, label = 'Median',color='green')
plt.bar(x_axis +0.05, e80_, width=bar_w, label = '80-%ile',color='cyan')
plt.bar(x_axis +0.15, e90_, width=bar_w, label = '90-%tile',color='black')

plt.xticks(x_axis, AggPes.keys(),rotation = 70)
plt.legend(bbox_to_anchor=(0.41,0.65))
plt.ylabel('Agg WAPE')
plt.xlabel('Model')
plt.show()

In [ ]:
TimesL=[]
AggSmapeL=[]
AggWapeL=[]
AggPeL=[]
times_org=[]
wape_agg_lstm=np.zeros(50, dtype=np.float64)
df_lstm=np.zeros((30,50), dtype=np.float64)

In [ ]:
start_month=['2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01',
            '2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01']
name=['September','October','November','December','January','February','March','April','May','June','July','August']

In [ ]:
month=1
print('Running for the month of :',name[month])
################LSTM##########################
################LSTM##########################
for i in range(50):
    print('cluster id: ',i+1)
    current_date = '20220831'
    test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
    #for top cluster:
    path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
    #path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
    # read in data
    df_panda = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
    df_panda=df_panda[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
    #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
    Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
    for cols in Cols:  
        df_panda[cols]=df_panda[cols].astype(float,errors='raise')
    start=time.time()
    start=time.time()
    minError=1e10
    param = {
        "learning_rate": [.1,.01,.001],
        "hidden_size": [10,20],
        'batch_size': [32],
        'dropout': [0.05,.1],
        'gamma': [0.9],
        'optimizer_name': ['AdamW'],
        'num_layer': [1,2,3],
        'relu': [False]
    }
    prep = dataprep(df_panda, input_window=90, output_window=30, stride=1,start=start_month[month])
    print('Beginning the training...')
    train_set_in, train_set_out, val_set_in, val_set_out, test_in_set, test_out_set,df_test_org,  q,wq = prep._get_data()
    #print('print len of q, value and wq value',len(q),q[0],wq[0])
    train_set_in = torch.from_numpy(train_set_in.astype(np.float64)).float()
    train_set_out = torch.from_numpy(train_set_out.astype(np.float64)).float()
    val_set_in = torch.from_numpy(val_set_in.astype(np.float64)).float()
    val_set_out = torch.from_numpy(val_set_out.astype(np.float64)).float()
    test_in_set = torch.from_numpy(test_in_set.astype(np.float64)).float()
    test_out_set = torch.from_numpy(test_out_set.astype(np.float64)).float()
    print('val shape is: ',val_set_in.shape, val_set_out.shape)
    end1=time.time()
    if len(train_set_in.shape) < 3:
        train_set_in = train_set_in.reshape(train_set_in.shape[0], train_set_in.shape[1], 1)
        val_set_in = val_set_in.reshape(val_set_in.shape[0], val_set_in.shape[1], 1)
        test_in_set = test_in_set.reshape(test_in_set.shape[0], test_in_set.shape[1], 1)
        test_out_set=test_out_set.reshape(test_out_set.shape[0], test_out_set.shape[1])
    counter=1
    for lr in param['learning_rate']:
        for h_s in param['hidden_size']:
            for b_s in param['batch_size']:
                for dp in param['dropout']:
                    for opt in param['optimizer_name']:
                        for n_l in param['num_layer']:
                            for act in param['relu']:
                                for gm in param['gamma']:
                                    config = {
                                        "num_epochs": 200,
                                        "learning_rate": lr,
                                        "hidden_size": h_s,
                                        "window_size": 30,
                                        "output_size": 30,
                                        'batch_size': b_s,
                                        'dropout': dp,
                                        'dropout1': 0,
                                        'gamma': gm,
                                        'optimizer_name': opt,
                                        'num_layer': n_l,
                                        'relu': act
                                    }
                                    #torch.manual_seed(seed)
                                    print('trial run: ',counter)
                                    counter+=1
                                    e1,e2,e3 = running_lstm(config, save_model=True)
                                    if (e1+e2+e3)/3<minError:
                                        minError=(e1+e2+e3)/3
                                        best_config=config
                                        print('the  smape is {} wape is {} and pe is {} and total is {}'.format(e1,e2,e3,(e1+e2+e3)/3))
                                        print('the config is {}',config)
    if minError>=.7:
        param = {
        "learning_rate": [.005,.1,.0001],
        "hidden_size": [15,30],
        'batch_size': [64],
        'dropout': [0.3,.5],
        'gamma': [0.9],
        'optimizer_name': ['Adam','AdamW'],
        'num_layer': [3,5],
        'relu': [True]}
        for lr in param['learning_rate']:
            for h_s in param['hidden_size']:
                for b_s in param['batch_size']:
                    for dp in param['dropout']:
                        for opt in param['optimizer_name']:
                            for n_l in param['num_layer']:
                                for act in param['relu']:
                                    for gm in param['gamma']:
                                        config = {
                                            "num_epochs": 200,
                                            "learning_rate": lr,
                                            "hidden_size": h_s,
                                            "window_size": 30,
                                            "output_size": 30,
                                            'batch_size': b_s,
                                            'dropout': dp,
                                            'dropout1': 0,
                                            'gamma': gm,
                                            'optimizer_name': opt,
                                            'num_layer': n_l,
                                            'relu': act
                                        }
                                        print('trial run: ',counter)
                                        counter+=1
                                        e1,e2,e3 = running_lstm(config, save_model=True)
                                        
                                        #print('the config is {}',config)
                                        if (e1+e2+e3)/3<minError:
                                            minError=(e1+e2+e3)/3
                                            best_config=config
                                            print('the  smape is {} wape is {} and pe is {} and total is {}'.format(e1,e2,e3,(e1+e2+e3)/3))
                                            print('the config is {}',config)

    end = time.time()
    print('thus best config is', best_config)
    Wsmape, Wape, Wpe, smape_dis, wape_dis, pe_dis, smape_agg, wape_agg, pe_agg, model, df_lstm[:,i],wape_agg_lstm[i] = train_lstm(best_config, save_model=True)
    print('val wape is: ',wape_agg_lstm[i])
    print('total time ', end1 - end)
    print('test Wsmape: ', Wsmape, ' test Wape: ', Wape, ' test Wpe: ', Wpe)
    print('Smape dis',smape_dis,'Wape dis',wape_dis,'PE dis',pe_dis)
    print('Agg SMAPE: ',smape_agg,'Agg WAPE: ',wape_agg,'Agg PE: ',pe_agg)
    print('total time ', end1 - end)

    #print('time needed to run the model: ',end-start)
    AggSmapeL.append(smape_agg)
    AggWapeL.append(wape_agg)
    AggPeL.append(pe_agg)
    SMAPEsL.append(Wsmape)
    WAPEsL.append(Wape)
    PEsL.append(Wpe)
    median_smape.append(smape_dis[0])
    Eighty_per_smape.append(smape_dis[1])
    Ninty_per_smape.append(smape_dis[2])
    median_wape.append(wape_dis[0])
    Eighty_per_wape.append(wape_dis[1])
    Ninty_per_wape.append(wape_dis[2])
    median_pe.append(pe_dis[0])
    Eighty_per_pe.append(pe_dis[1])
    Ninty_per_pe.append(pe_dis[2])
    TimesL.append(end1-end)
    times_org.append(end1-start)
    print('===============================================')
    
    

In [ ]:
Times=[]
AggSmape=[]
AggWape=[]
AggPe=[]

counter=0
for i in range(50):
    if i not in [44,47,48]:
        
        st=time.time()
        print('cluster id: ',i+1)
        current_date = '20220831'
        test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
        print('group id',test_group_id)
        #for top cluster:
        #path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
        path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
        # read in data
        df = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
        df=df[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
        #print('query id: ',df['query_idx'].unique())
        #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
        Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
        for cols in Cols:df[cols]=df[cols].astype(float,errors='raise')
        #print('time needed to preprocess the data is: ',time.time()-st)
        df_train,df_test,train_date,test_date,q,wq=_generate_train_test_data(df,start=start_month[month])
        df_trainW,df_valW,_,train_dateW,_,_,_=_generate_train_test_data(df,val_req=True,start=start_month[month])
        df1_train,df1_test=df_train.sum(axis=1),df_test.sum(axis=1)
        df1_trainW,df_valW=df_trainW.sum(axis=1),df_valW.sum(axis=1)
        df1_train,df1_trainW,df_valW=df1_train.to_frame(),df1_trainW.to_frame(),df_valW.to_frame()
        df1_train.index=train_date
        df1_trainW.index=train_dateW
        df1_train.index = df1_train.index.to_period(freq = 'D')
        df1_trainW.index = df1_trainW.index.to_period(freq = 'D')
        #df_test_log_temp.index = df_test_log_temp.index.to_period(freq = 'D')
        df1_train=df1_train.rename(columns={0:'daily_supply'})
        df1_trainW=df1_trainW.rename(columns={0:'daily_supply'})
    #     if i==0:
    #         print(q,wq)
        start=time.time()
        Models=statModels()

        Models.clus_smape['Lstm']=AggSmapeL[counter]
        Models.clus_wape['Lstm']=AggWapeL[counter]
        Models.clus_pe['Lstm']=AggPeL[counter]
        #Models.models['Lstm'] = fit4
        #self.stores_mape['Es'] = mape
        Models.stores_times['Lstm'] = TimesL[counter]
        ##################
        #Models.wape_Weighted['Lstm'] = wape_agg_lstm[i]

        Models.training_infer(df1_train,df1_test,df_test,q.values,wq.values,df1_test.shape[0],df_lstm[:,i])#no_ts
                             #(df_train, df_test, df_test_org,q, wq, forecast_period=30,lstm_pred=None)
        Models.weighted_avg(df1_trainW, df1_test,df_valW.values, df_test,q.values,wq.values, forecast_period=60,lstm_pred=df_lstm[:,i])

        end=time.time()
        print('Wape is: ',Models.clus_wape)
        print('Smape is: ',Models.clus_smape)
        print('Pe is: ',Models.clus_smape)
        print('Time is:' ,Models.clus_pe)
        #print('time needed to run the model: ',end-start)
        Times.append(Models.stores_times)
        AggSmape.append(Models.clus_smape)
        AggWape.append(Models.clus_wape)
        AggPe.append(Models.clus_pe)
        #training_j.append(i)
        counter+=1
    print('===============================================')
    

In [ ]:

Time={i:[] for i in Times[0]}

AggSmapes={i:[] for i in AggSmape[0]}
AggWapes={i:[] for i in AggWape[0]}
AggPes={i:[] for i in AggPe[0]}


for i in range(counter):

    for keys,values in Times[i].items():
        Time[keys].append(values)
    for keys,values in AggSmape[i].items():
        AggSmapes[keys].append(values)
    for keys,values in AggWape[i].items():
        AggWapes[keys].append(values)
    for keys,values in AggPe[i].items():
        AggPes[keys].append(values)
    
print('Time is: ',Time)
print('====================')
print('AggSmapes is: ',AggSmapes)
print('====================')
print('AggWapes is: ',AggWapes)
print('====================')
print('AggPes is: ',AggPes)
print('====================')

print('====================')
print('Original LSTM time is', times_org)    

In [ ]:
TimesL=[]
AggSmapeL=[]
AggWapeL=[]
AggPeL=[]
times_org=[]
wape_agg_lstm=np.zeros(50, dtype=np.float64)
df_lstm=np.zeros((30,50), dtype=np.float64)

In [ ]:
start_month=['2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01',
            '2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01']
name=['September','October','November','December','January','February','March','April','May','June','July','August']

In [ ]:
month=2
print('Running for the month of :',name[month])
################LSTM##########################
################LSTM##########################
for i in range(50):
    print('cluster id: ',i+1)
    current_date = '20220831'
    test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
    #for top cluster:
    path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
    #path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
    # read in data
    df_panda = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
    df_panda=df_panda[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
    #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
    Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
    for cols in Cols:  
        df_panda[cols]=df_panda[cols].astype(float,errors='raise')
    start=time.time()
    start=time.time()
    minError=1e10
    param = {
        "learning_rate": [.1,.01,.001],
        "hidden_size": [10,20],
        'batch_size': [32],
        'dropout': [0.05,.1],
        'gamma': [0.9],
        'optimizer_name': ['AdamW'],
        'num_layer': [1,2,3],
        'relu': [False]
    }
    prep = dataprep(df_panda, input_window=90, output_window=30, stride=1,start=start_month[month])
    print('Beginning the training...')
    train_set_in, train_set_out, val_set_in, val_set_out, test_in_set, test_out_set,df_test_org,  q,wq = prep._get_data()
    #print('print len of q, value and wq value',len(q),q[0],wq[0])
    train_set_in = torch.from_numpy(train_set_in.astype(np.float64)).float()
    train_set_out = torch.from_numpy(train_set_out.astype(np.float64)).float()
    val_set_in = torch.from_numpy(val_set_in.astype(np.float64)).float()
    val_set_out = torch.from_numpy(val_set_out.astype(np.float64)).float()
    test_in_set = torch.from_numpy(test_in_set.astype(np.float64)).float()
    test_out_set = torch.from_numpy(test_out_set.astype(np.float64)).float()
    print('val shape is: ',val_set_in.shape, val_set_out.shape)
    end1=time.time()
    if len(train_set_in.shape) < 3:
        train_set_in = train_set_in.reshape(train_set_in.shape[0], train_set_in.shape[1], 1)
        val_set_in = val_set_in.reshape(val_set_in.shape[0], val_set_in.shape[1], 1)
        test_in_set = test_in_set.reshape(test_in_set.shape[0], test_in_set.shape[1], 1)
        test_out_set=test_out_set.reshape(test_out_set.shape[0], test_out_set.shape[1])
    counter=1
    for lr in param['learning_rate']:
        for h_s in param['hidden_size']:
            for b_s in param['batch_size']:
                for dp in param['dropout']:
                    for opt in param['optimizer_name']:
                        for n_l in param['num_layer']:
                            for act in param['relu']:
                                for gm in param['gamma']:
                                    config = {
                                        "num_epochs": 200,
                                        "learning_rate": lr,
                                        "hidden_size": h_s,
                                        "window_size": 30,
                                        "output_size": 30,
                                        'batch_size': b_s,
                                        'dropout': dp,
                                        'dropout1': 0,
                                        'gamma': gm,
                                        'optimizer_name': opt,
                                        'num_layer': n_l,
                                        'relu': act
                                    }
                                    #torch.manual_seed(seed)
                                    print('trial run: ',counter)
                                    counter+=1
                                    e1,e2,e3 = running_lstm(config, save_model=True)
                                    if (e1+e2+e3)/3<minError:
                                        minError=(e1+e2+e3)/3
                                        best_config=config
                                        print('the  smape is {} wape is {} and pe is {} and total is {}'.format(e1,e2,e3,(e1+e2+e3)/3))
                                        print('the config is {}',config)
    if minError>=.7:
        param = {
        "learning_rate": [.005,.1,.0001],
        "hidden_size": [15,30],
        'batch_size': [64],
        'dropout': [0.3,.5],
        'gamma': [0.9],
        'optimizer_name': ['Adam','AdamW'],
        'num_layer': [3,5],
        'relu': [True]}
        for lr in param['learning_rate']:
            for h_s in param['hidden_size']:
                for b_s in param['batch_size']:
                    for dp in param['dropout']:
                        for opt in param['optimizer_name']:
                            for n_l in param['num_layer']:
                                for act in param['relu']:
                                    for gm in param['gamma']:
                                        config = {
                                            "num_epochs": 200,
                                            "learning_rate": lr,
                                            "hidden_size": h_s,
                                            "window_size": 30,
                                            "output_size": 30,
                                            'batch_size': b_s,
                                            'dropout': dp,
                                            'dropout1': 0,
                                            'gamma': gm,
                                            'optimizer_name': opt,
                                            'num_layer': n_l,
                                            'relu': act
                                        }
                                        print('trial run: ',counter)
                                        counter+=1
                                        e1,e2,e3 = running_lstm(config, save_model=True)
                                        
                                        #print('the config is {}',config)
                                        if (e1+e2+e3)/3<minError:
                                            minError=(e1+e2+e3)/3
                                            best_config=config
                                            print('the  smape is {} wape is {} and pe is {} and total is {}'.format(e1,e2,e3,(e1+e2+e3)/3))
                                            print('the config is {}',config)

    end = time.time()
    print('thus best config is', best_config)
    Wsmape, Wape, Wpe, smape_dis, wape_dis, pe_dis, smape_agg, wape_agg, pe_agg, model, df_lstm[:,i],wape_agg_lstm[i] = train_lstm(best_config, save_model=True)
    print('val wape is: ',wape_agg_lstm[i])
    print('total time ', end1 - end)
    print('test Wsmape: ', Wsmape, ' test Wape: ', Wape, ' test Wpe: ', Wpe)
    print('Smape dis',smape_dis,'Wape dis',wape_dis,'PE dis',pe_dis)
    print('Agg SMAPE: ',smape_agg,'Agg WAPE: ',wape_agg,'Agg PE: ',pe_agg)
    print('total time ', end1 - end)

    #print('time needed to run the model: ',end-start)
    AggSmapeL.append(smape_agg)
    AggWapeL.append(wape_agg)
    AggPeL.append(pe_agg)
    SMAPEsL.append(Wsmape)
    WAPEsL.append(Wape)
    PEsL.append(Wpe)
    median_smape.append(smape_dis[0])
    Eighty_per_smape.append(smape_dis[1])
    Ninty_per_smape.append(smape_dis[2])
    median_wape.append(wape_dis[0])
    Eighty_per_wape.append(wape_dis[1])
    Ninty_per_wape.append(wape_dis[2])
    median_pe.append(pe_dis[0])
    Eighty_per_pe.append(pe_dis[1])
    Ninty_per_pe.append(pe_dis[2])
    TimesL.append(end1-end)
    times_org.append(end1-start)
    print('===============================================')
    
    

In [ ]:
Times=[]
AggSmape=[]
AggWape=[]
AggPe=[]

counter=0
for i in range(50):
    if i not in [44,47,48]:
        
        st=time.time()
        print('cluster id: ',i+1)
        current_date = '20220831'
        test_group_id = test_group_id_list[i]#'/3944/1060825/447913'
        print('group id',test_group_id)
        #for top cluster:
        #path = f"gs://adtech-artifacts/ds/forecast/month_eval/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_tft_data_{current_date}.csv"
        path = f"gs://adtech-artifacts/ds/forecast/month_eval/tail_clusters/query_taxonomy_clusters_sample{'_'.join(test_group_id.split('/'))}_data_20220831.csv"
        # read in data
        df = spark.read.option("header", True).csv(path).toPandas() # The usage should be the same as previous
        df=df[['ds', 'group_id', 'query_idx', 'SearchKeyValue',  'LocationKeyValue', 'daily_supply']]
        #print('query id: ',df['query_idx'].unique())
        #df_panda=spark.read.csv("gs://adtech-artifacts/ds/forecast/query_taxonomy_sample_Addedfeatures_cluster_"+str(i)+".csv",header=True).toPandas()
        Cols=['daily_supply']#, 'shifted_annual_daily_supply','Trend_Var','dayofyear_sin', 'dayofyear_cos', 'month_sin', 'month_cos']
        for cols in Cols:df[cols]=df[cols].astype(float,errors='raise')
        #print('time needed to preprocess the data is: ',time.time()-st)
        df_train,df_test,train_date,test_date,q,wq=_generate_train_test_data(df,start=start_month[month])
        df_trainW,df_valW,_,train_dateW,_,_,_=_generate_train_test_data(df,val_req=True,start=start_month[month])
        df1_train,df1_test=df_train.sum(axis=1),df_test.sum(axis=1)
        df1_trainW,df_valW=df_trainW.sum(axis=1),df_valW.sum(axis=1)
        df1_train,df1_trainW,df_valW=df1_train.to_frame(),df1_trainW.to_frame(),df_valW.to_frame()
        df1_train.index=train_date
        df1_trainW.index=train_dateW
        df1_train.index = df1_train.index.to_period(freq = 'D')
        df1_trainW.index = df1_trainW.index.to_period(freq = 'D')
        #df_test_log_temp.index = df_test_log_temp.index.to_period(freq = 'D')
        df1_train=df1_train.rename(columns={0:'daily_supply'})
        df1_trainW=df1_trainW.rename(columns={0:'daily_supply'})
    #     if i==0:
    #         print(q,wq)
        start=time.time()
        Models=statModels()

        Models.clus_smape['Lstm']=AggSmapeL[counter]
        Models.clus_wape['Lstm']=AggWapeL[counter]
        Models.clus_pe['Lstm']=AggPeL[counter]
        #Models.models['Lstm'] = fit4
        #self.stores_mape['Es'] = mape
        Models.stores_times['Lstm'] = TimesL[counter]
        ##################
        #Models.wape_Weighted['Lstm'] = wape_agg_lstm[i]

        Models.training_infer(df1_train,df1_test,df_test,q.values,wq.values,df1_test.shape[0],df_lstm[:,i])#no_ts
                             #(df_train, df_test, df_test_org,q, wq, forecast_period=30,lstm_pred=None)
        Models.weighted_avg(df1_trainW, df1_test,df_valW.values, df_test,q.values,wq.values, forecast_period=60,lstm_pred=df_lstm[:,i])

        end=time.time()
        print('Wape is: ',Models.clus_wape)
        print('Smape is: ',Models.clus_smape)
        print('Pe is: ',Models.clus_smape)
        print('Time is:' ,Models.clus_pe)
        #print('time needed to run the model: ',end-start)
        Times.append(Models.stores_times)
        AggSmape.append(Models.clus_smape)
        AggWape.append(Models.clus_wape)
        AggPe.append(Models.clus_pe)
        #training_j.append(i)
        counter+=1
    print('===============================================')
    

In [ ]:

Time={i:[] for i in Times[0]}

AggSmapes={i:[] for i in AggSmape[0]}
AggWapes={i:[] for i in AggWape[0]}
AggPes={i:[] for i in AggPe[0]}


for i in range(counter):

    for keys,values in Times[i].items():
        Time[keys].append(values)
    for keys,values in AggSmape[i].items():
        AggSmapes[keys].append(values)
    for keys,values in AggWape[i].items():
        AggWapes[keys].append(values)
    for keys,values in AggPe[i].items():
        AggPes[keys].append(values)
    
print('Time is: ',Time)
print('====================')
print('AggSmapes is: ',AggSmapes)
print('====================')
print('AggWapes is: ',AggWapes)
print('====================')
print('AggPes is: ',AggPes)
print('====================')

print('====================')
print('Original LSTM time is', times_org)    